<center><br> <img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 300px;"></center><br><br>

<img src="https://upload.wikimedia.org/wikipedia/en/archive/c/cc/20180621135549%21Vodafone_2017_logo.svg" style="width: 200px;margin-top:90px;margin-bottom:100px;" align="left">
   <div style= "font-size: 28px;font-weight:bold;margin-top:20px;margin-bottom:20px;margin-left:360px;margin-right:150px; line-height: 1.1;color:#F91701;"><center>Análise de Sentimentos nas Redes Sociais | Vodafone</center></div>
   <div style= "font-size: 17px;font-weight:bold;"><center>Projeto Final Aplicado em Ciência de Dados</center></div>
   <div><center><b>Orientador:</b> Nuno Santos (ISCTE-IUL) </center></div>
   <div><center><b>Cliente:</b> Carlos Santos (Vodafone) </center></div>
 <br>
    <div><center>André Silvestre Nº104532 </center></div>
    <div><center>Eliane Gabriel Nº103303 <b>|</b> Maria João Lourenço Nº104716 </center></div>
    <div><center>Margarida Pereira Nº105877 <b>|</b> Umeima Mahomed Nº99239</center></div>
    <br>
    <div><center><b>CDC1</b> & <b>CDC2</b></center></div>
 <br>

<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: .7px; color: white; border-radius: 300px; text-align: center;">
</div>

# Índice


<ol style="list-style-type: upper-roman;font-weight: bold;">
  <li>
    <span style="font-weight: normal;">
      <a href="">Business Understanding</a>
    </span> 
  </li> 
    
   <li>
    <span style="font-weight: normal;">
      <a href="">Data Understanding</a>
    </span> 
  </li> 
    
  <li>
    <span style="font-weight: normal;">
      <a href="#2">Data Preparation</a>
    </span> 
    <ol style="list-style-type:decimal;">
      <li><span style="font-weight: normal;"><a href="#2.4">Pré-Processamento de Texto</a></span></li>
    </ol>
  </li>
  <li>
    <span style = "font-weight: normal;">
        <a href="#3">Modeling</a>
    </span>
    <ol style="list-style-type:decimal;">
      <li><span style="font-weight: normal;"><a href="#4.1">Sentiment Analysis</a></span></li>
      <li><span style="font-weight: normal;"><a href="#4.2">Topic Analysis</a></span></li>
    </ol>
   </li>
  <li><span style = "font-weight: normal;"><a href="#5">Evaluation</a></span></li>
  <li><span style = "font-weight: normal;"><a href="#6">Deployment [Dashboard]</a></span></li>
</ol>

<br>
<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: .7px; color: white; border-radius: 300px; text-align: center;">
</div>
<br>

In [1]:
from IPython.core.display import HTML
HTML("""<style>ol > li::marker {font-weight: bold;font-size: 15px;}
    .output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>""")

In [2]:
# Fonte: https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args, super_title, titles=cycle([''])):
    html_str = ''
    html_str += f'<h1 style="text-align: center; margin-bottom: -50px;">{super_title}</h1><br>'
    html_str += '<div style="display: flex; justify-content: center; align-items: center;">'
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
        html_str += f'<div style="margin-right: 20px;"><h3 style="text-align: center;color:#555555; margin-bottom: 10px">{title}</h3>'
        html_str += df.to_html().replace('table', 'table style="display:inline; margin-right: 20px;"')
        html_str += '</div>'
    html_str += '</div>'
    display_html(html_str, raw=True)

## 📚 Import das Bibliotecas

In [3]:
# Bibliotecas que vamos usar no Projeto
import os                # Operating System
from tqdm import tqdm    # Barra de Progresso

# Para dados
import pandas as pd
import numpy as np
import json
import re
import time
import datetime
import unidecode
from tqdm import tqdm                        # Barra de Progresso
pd.set_option('display.max_colwidth', None)  # Visualizar a informação toda
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_colwidth') 

# Para gráficos
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
import seaborn as sns
import datetime

# -------------------- Text Mining -------------------
import collections
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt')
import spacy
import string
pd.options.mode.chained_assignment = None
import swifter

# ---------------- Sentiment/Topic Analysis ----------------
from tqdm.auto import tqdm     # Barra de Progressos em pipelines
tqdm.pandas()

# Estilo dos Gráficos    
plt.style.use('ggplot')
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
sns.set_style("white")

# Semente para reprodutibilidade
import random
random.seed(123)

In [4]:
# Confirmar o environment do Anaconda
import subprocess
print(subprocess.check_output(["conda", "info", "--env"], universal_newlines=True))

---

# <a class='anchor' id='2'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>3 | Data Preparation</b></h1></center>
</div>

## 🧮 Importar as Bases de Dados dos Posts, Comentários e Users

In [5]:
Facebook_Posts = pd.read_csv('Datasets_Vodafone/Facebook_Posts.txt', sep='\t', encoding='utf-8')
Facebook_Comments = pd.read_csv('Datasets_Vodafone/Facebook_Comments.txt', sep='\t', encoding='utf-8')
Facebook_Users = pd.read_csv('Datasets_Vodafone/Facebook_Users.txt', sep='\t', encoding='utf-8')

# Garantir que as colunas 'post_id' e 'page' e 'user_link' são do tipo str
Facebook_Users['user_link'] = Facebook_Users['user_link'].astype(str)

Facebook_Comments['post_id'] = Facebook_Comments['post_id'].astype(str)
Facebook_Comments['page'] = Facebook_Comments['page'].astype(str)

Facebook_Posts['post_id'] = Facebook_Posts['post_id'].astype(str)
Facebook_Posts['page'] = Facebook_Posts['page'].astype(str)

In [6]:
# Juntar tabelas e restringir aos anos em estudo
Facebook_Posts_Comments = pd.merge(Facebook_Comments, Facebook_Posts, how='outer', on=['post_id', 'page'])

# Filtrar o DataFrame Facebook_Posts para obter os posts a partir de 2019
Facebook_Posts = Facebook_Posts[Facebook_Posts['post_year'] >= 2019]

# Filtrar o DataFrame Facebook_Comments para obter os posts a partir de 2019
Facebook_Comments = Facebook_Posts_Comments[Facebook_Posts_Comments['post_year'] >= 2019]

In [7]:
print("\033[1mNº de Publicações entre 2019-2024:\033[0m {:>12}".format(len(Facebook_Posts)))
print("\033[1mNº de Comentários entre 2019-2024:\033[0m {:>14}".format(len(Facebook_Comments)))
print("\033[1mNº de Utilizadores Únicos entre 2019-2024:\033[0m {:>3}".format(len(
    Facebook_Comments[~Facebook_Comments.duplicated(subset=['user_link'])])))

# <a class='anchor' id='2.4'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: transparent; 
            padding: 10px; color: white; border-radius: 300px; text-align: center;
            border: 2px solid #A30000;">
    <center><h2 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: #A30000;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>Pré-Processamento de Texto</b></h2></center>
</div>

In [8]:
# Verificar posts sem texto
print('\033[1mNº de Posts a eliminar:\033[0m', len(Facebook_Posts[Facebook_Posts['post_text'].isna()]),
      '(', round(len(Facebook_Posts[Facebook_Posts['post_text'].isna()])/len(Facebook_Posts)*100, 2),'%)')

Facebook_Posts['post_text'] = Facebook_Posts['post_text'].fillna(' ')

> Não vamos eliminar pq se não elimina tmb os comentários associados.

In [9]:
# Verificar comentários sem texto [com emojis/só fotos/...]
print('\033[1mNº de Comentários a eliminar:\033[0m', len(Facebook_Comments[Facebook_Comments['comment_text'].isna()]),
      '(', round(len(Facebook_Comments[Facebook_Comments['comment_text'].isna()])/len(Facebook_Comments)*100, 2),'%)')

# Eliminar NAs
Facebook_Comments.dropna(subset=['comment_text'], inplace=True)

# Verificar
print('\033[1mNº de Comentários a eliminar:\033[0m', len(Facebook_Comments[Facebook_Comments['comment_text'].isna()]),
      '(', round(len(Facebook_Comments[Facebook_Comments['comment_text'].isna()])/len(Facebook_Comments)*100, 2),'%)')

In [10]:
# Verificar todos os DataFrames quanto à presença do caractere '�'
for df_name, df in zip(['Facebook_Posts', 'Facebook_Comments'], [Facebook_Posts, Facebook_Comments]):
    print(f"\033[1mVerificando o DataFrame {df_name}...\033[0m")
    for column in df.columns:
        contains_character = df[column].astype(str).str.contains('�', na=False).any()
        if contains_character:
            print(f"Caractere '�' encontrado na coluna '{column}' do DataFrame {df_name}")

In [11]:
# Verificar as linhas com caracter especial '�'
CE_Comment_Text = Facebook_Comments[Facebook_Comments['comment_text'].str.contains('�')]
print('\033[1mTotal Observações de Comentários com Caracter Especial = \033[0m', len(CE_Comment_Text),
     '(',round(len(CE_Comment_Text)/len(Facebook_Comments)*100,5),'%)')
# CE_Comment_Text - Verificou-se que também se encontrava assim no comentário original

# Remover os caracteres '�' substituindo-os por uma string vazia ''
Facebook_Comments['comment_text'] = Facebook_Comments['comment_text'].str.replace('�', '')

# Verificar se a substituição foi bem-sucedida
print('\033[1mTotal Observações de Comentários com Caracter Especial = \033[0m', 
      len(Facebook_Comments[Facebook_Comments['comment_text'].str.contains('�')]))

In [12]:
# Redefinir o nº de linhas
Facebook_Posts = Facebook_Posts.reset_index(drop=True)
Facebook_Comments = Facebook_Comments.reset_index(drop=True)

---

In [13]:
# Verificar as linhas a expressão "… Ver mais"
VM_Post_Text = Facebook_Posts[Facebook_Posts['post_text'].str.contains('… Ver mais')]
print('\033[1mTotal Observações com "… Ver mais" = \033[0m', len(VM_Post_Text),
     '(',round(len(VM_Post_Text)/len(Facebook_Posts)*100,1),'%)')
# CE_Post_Text - Verificou-se que também se encontrava assim no comentário original

In [14]:
# Verificar as linhas a expressão "… Ver mais"
VM_Comment_Text = Facebook_Comments[Facebook_Comments['comment_text'].str.contains('… Ver mais')]
print('\033[1mTotal Observações com "… Ver mais" = \033[0m', len(VM_Comment_Text),
     '(',round(len(VM_Comment_Text)/len(Facebook_Comments)*100,1),'%)')
# CE_Comment_Text - Verificou-se que também se encontrava assim no comentário original

In [15]:
# Apagar a expressão "… Ver mais" ============= POSTS
Facebook_Posts['post_text'] = Facebook_Posts['post_text'].str.replace('… Ver mais', '')

# Verificar se a substituição foi bem-sucedida
print('\033[1mTotal Observações com "… Ver mais" = \033[0m', 
      len(Facebook_Posts[Facebook_Posts['post_text'].str.contains('… Ver mais')]))

In [16]:
# Apagar a expressão "… Ver mais" ============= COMMENTS
Facebook_Comments['comment_text'] = Facebook_Comments['comment_text'].str.replace('… Ver mais', '')

# Verificar se a substituição foi bem-sucedida
print('\033[1mTotal Observações com "… Ver mais" = \033[0m', 
      len(Facebook_Comments[Facebook_Comments['comment_text'].str.contains('… Ver mais')]))

---

### 💬 Identificação da Linguagem dos Texto

In [17]:
# !pip install lingua-language-detector

In [18]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH, Language.PORTUGUESE]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

# Função para identificar o idioma do texto
def detect_language(text):
    try:
        return detector.detect_language_of(text)
    except:
        return 'Unknown'
    
# Identificar idioma para 'post_text'
Facebook_Posts['post_language'] = Facebook_Posts['post_text'].swifter.apply(detect_language)

In [19]:
# Identificar idioma para 'comment_text'
Facebook_Comments['comment_language'] = Facebook_Comments['comment_text'].swifter.apply(detect_language)

In [20]:
# Frequência de linguagens dos 'post_text' e 'comment_language'
post_lang_freq = Facebook_Posts['post_language'].value_counts()
comments_lang_freq = Facebook_Comments['comment_language'].value_counts().fillna(0).astype(int)

# Tabela de frequência absoluta e relativa
lang_freq = pd.DataFrame({'Publicações (n)': post_lang_freq,
                          'Publicações (%)': round(post_lang_freq / post_lang_freq.sum()*100,1),
                          'Comentários (n)': comments_lang_freq,
                          'Comentários (%)':round(comments_lang_freq / comments_lang_freq.sum()*100,1)})
lang_freq.index.name = 'Linguagens'
lang_freq.rename(index={Language.PORTUGUESE: 'Português', Language.ENGLISH: 'Inglês', 'Unknown':'NI'}, inplace=True)
lang_freq = lang_freq.reindex(index=['Português', 'Inglês', 'NI']) # NI = Não Identificado
lang_freq = lang_freq.fillna(0)
lang_freq['Publicações (n)'] = lang_freq['Publicações (n)'].astype(int)
lang_freq['Comentários (n)'] = lang_freq['Comentários (n)'].astype(int)
lang_freq

In [21]:
# Facebook_Posts[(Facebook_Posts['post_language'] != Language.PORTUGUESE) &
#                (Facebook_Posts['post_language'] != 'Unknown')
#               ][['post_text', 'post_language']]

In [22]:
# Facebook_Comments[Facebook_Comments['comment_language'] == Language.ENGLISH]

---

### 📚 Tradução do Texto | HuggingFace [Translation Pipeline](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TranslationPipeline)

**Fonte:** Workshop *Hands-on Transformers: Fine-tune your own BERT and GPT* (Moritz Laurer from HuggingFace) [Github Code](https://github.com/MoritzLaurer/summer-school-transformers-2023/blob/main/)

**Machine Translation**

* Open source machine translation (MT) models enable you to translate between many different languages without Google Translate.
* [University of Helsinki](https://huggingface.co/Helsinki-NLP) uploaded models for more than 1000 language pairs to the Hugging Face hub
* [Facebook AI](https://huggingface.co/models?search=facebook+m2m) open-sourced several multi-lingual models
* The [EasyNMT library](https://github.com/UKPLab/EasyNMT), provides an easy wrapper for all these models
* Most machine translation models translate between two languages in one direction (e.g. German to English, but not English to German), some can translate in multiple directions.

In [21]:
# !pip install transformers
# The Transformers library from Hugging Face
# !pip install sentencepiece
# optional tokeniser, required for some models. e.g. machine translation
# running large models with accelerate https://huggingface.co/blog/accelerate-large-models
# NOTE: we need to restart the runtime after installing accelerate
# !pip install accelerate
############# https://pytorch.org/ -> Instalar

In [16]:
# automatically chose CPU or GPU for inference, depending on your hardware
import torch
device = torch.cuda.current_device() if torch.cuda.is_available() else -1
# -1 == CPU ; 0 == GPU
print(device)

In [17]:
import os
os.environ['HF_HOME'] = 'C:/HuggingFace'               #   -----------> Caso de Erro -> Alterar a paste de '.cache'
os.environ['HF_DATASETS_CACHE'] = 'C:/HuggingFace'
os.environ['TRANSFORMERS_CACHE'] = 'C:/HuggingFace'

In [22]:
# ```Python
from transformers import pipeline

# translation pipeline docs: https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TranslationPipeline
pipeline_translate = pipeline("translation",                   # Tipo de pipeline
                              model="facebook/m2m100_418M",    # Modelo
                              tokenizer="facebook/m2m100_418M",
                              # framework='pt',                  # 'pt' for PyTorch or 'tf' for TensorFlow
                              device=device,                    # -1 -> CPU | 0 -> CUDA [NVIDIA]
                              batch_size=8
                             )

# Função para traduzir um texto
def translate_text(text):
    # Aplicar o pipeline de tradução ao texto
    translated_text = pipeline_translate(text, src_lang="pt", tgt_lang="en")[0]["translation_text"]
    return translated_text
# ```

In [23]:
# Frase Teste
text_example = "As operadoras de telecomunicações oferecem uma gama de serviços de comunicação."
translate_text(text_example)

In [25]:
# # Aplicar a tradução aos posts e comentários do Facebook
# Facebook_Posts['post_text_english'] = Facebook_Posts['post_text'].progress_apply(translate_text)

In [26]:
# # Aplicar a tradução aos posts e comentários do Facebook
# Facebook_Comments['comment_text_english'] = Facebook_Comments['comment_text'].progress_apply(translate_text)

```Python

# Aplicar a tradução aos posts e comentários do Facebook
Facebook_Posts['post_text_english'] = Facebook_Posts['post_text'].progress_apply(translate_text)

# Salvar a nova coluna como um arquivo CSV
Facebook_Posts[['post_id', 'post_text_english']].to_csv('Datasets_Vodafone/TextMining/post_text_english.csv', encoding='utf-8', index=False)

# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
post_text_english = pd.read_csv('post_text_english.csv')
Facebook_Posts = Facebook_Posts.merge(post_text_english, on='post_id', how='left')
```

```Python
# Aplicar a tradução aos posts e comentários do Facebook
Facebook_Comments['comment_text_english'] = Facebook_Comments['comment_text'].progress_apply(translate_text)

# Salvar a nova coluna como um arquivo CSV
Facebook_Comments[['comment_id', 'comment_text_english']].to_csv('Datasets_Vodafone/TextMining/comment_text_english.csv', encoding='utf-8', index=False)

# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
comment_text_english = pd.read_csv('comment_text_english.csv')
Facebook_Comments = Facebook_Comments.merge(comment_text_english, on='comment_id', how='left')
```

> Por questões de **eficiência** e **possíveis perdas de informação** útil à análise de sentimentos no texto, ***optou-se por não traduzir o texto*** e utilizar o original na língua que o utilizador comentou

---

## 🛠️ Pré-Processamento do Texto

**Fonte:** https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing


> **NLTK** is the most famous Python Natural Language Processing Toolkit.

Algumas das etapas comuns de pré-processamento / limpeza de texto são:

* *Lower casing*             ✅
* *Removal of Punctuations*  ✅
* *Removal of URLs*          ✅
* *Removal Extra Spaces*     ✅
* *Chat words conversion*    ✅
* *Spelling correction*      ✅

Estes são os diferentes tipos de etapas de pré-processamento de texto que se pode efetuar nos dados de texto. Mas não é necessário fazer tudo isto sempre. É importante escolher cuidadosamente as etapas de pré-processamento com base no nosso caso de utilização, uma vez que isso também desempenha um papel importante.

### Lower Casing

***Lower Casing*** é uma técnica comum de pré-processamento de texto. A ideia é converter o texto de *input* para o mesmo formato, de modo a que `'texto'`, `'Texto'` e `'TEXTO'` sejam tratados da mesma forma.

Isto é mais útil para as técnicas de caraterização do texto, como a frequência e o `tfidf`, uma vez que ajuda a combinar as mesmas palavras, reduzindo assim a duplicação e obtendo contagens/valores `tfidf` correctos.

Por padrão, o lower casing é feito pela maioria dos *vecotirzers* e *tokenizers* modernos, como o **`sklearn`** `TfidfVectorizer` e o `Keras Tokenizer`. Por isso, é preciso de os definir como falsos conforme necessário, dependendo do caso de utilização.

---

In [23]:
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text'].str.lower()
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text'].str.lower()

In [24]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

### Removal of URLs

O próximo passo do pré-processamento é remover quaisquer URLs presentes nos dados. Por exemplo, se se estiver a fazer uma análise do Twitter, é muito provável que o tweet contenha algum URL. É provável que se precise de os remover para a análise posterior. 

In [25]:
def remove_urls(text):
    if text:
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.sub(r'', text)
    else:
        pass

# Teste da Função
text = "Boa tarde https://www.siteexemplo.com/"
print('\033[1mAntes:\033[0m', text)
print('\033[1mDepois:\033[0m', remove_urls(text))

In [26]:
# Aplicar ao dataset
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: remove_urls(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: remove_urls(text))

In [27]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

### Removal of Punctuations

Uma outra técnica comum de pré-processamento de texto consiste em remover as pontuações dos dados de texto. Trata-se, mais uma vez, de um processo de normalização do texto que ajudará a tratar 'Olá' e 'Olá!' da mesma forma.

Também é preciso escolher cuidadosamente a lista de pontuações a excluir, dependendo do caso de utilização. Por exemplo, o `string.punctuation` em python contém os seguintes símbolos de pontuação:

`!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`

Pode-se acrescentar ou retirar mais pontuações consoante as necessidades.

---

In [28]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    if text:
        """Custom function to remove the punctuation"""
        return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    else:
        pass

# Aplicar ao dataset
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: remove_punctuation(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: remove_punctuation(text))

In [29]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

### Removal Extra Spaces

A fase seguinte envolve retirar espaços extra que possam estar entre as palavras de uma frase. Para isso, vai ser criada a seguinte função: 

In [30]:
def remove_extra_spaces(text):
    if text:
        return re.sub(r'\s+', ' ', text)

# Teste
sentence = "Isso   é   um   exemplo    com    espaços   extra."
print("\033[1mAntes:\033[0m", sentence)
print("\033[1mDepois:\033[0m", remove_extra_spaces(sentence))

# Remover espaços | Aplicar ao dataset
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: remove_extra_spaces(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: remove_extra_spaces(text))

In [31]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

### Chat Words Conversion

Este é um passo importante de pré-processamento de texto se se estiver a lidar com dados de conversação. Os utilizadores usam muitas palavras abreviadas no chat e, por isso, pode ser útil expandir essas palavras para fins de análise. 

Lista de palavras de gíria inglesas de ***chat*** neste [Repositório](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt).

In [32]:
#!pip install deep_translator ---- Caso não usássemos o pipeline('translation')

In [33]:
# Fonte: https://medium.com/analytics-vidhya/how-to-translate-text-with-python-9d203139dcf5
from deep_translator import GoogleTranslator

to_translate = 'I want to translate this text'
print("\033[1mAntes:\033[0m", to_translate)
print("\033[1mDepois:\033[0m", GoogleTranslator(source='english', target='portuguese').translate(to_translate))

In [34]:
# # Função para traduzir um texto
# def translate_text_pt(text):
#     # Aplicar o pipeline de tradução ao texto
#     translated_text = pipeline_translate(text, src_lang="en", tgt_lang="pt")[0]["translation_text"]
#     return translated_text

In [35]:
# print("\033[1mCom o pipeline:\033[0m", translate_text_pt(to_translate))

In [36]:
file = open('Datasets_Vodafone/Auxiliares/abreviaturas_inglesas.txt','r', encoding='utf-8')
chat_words_str_en = file.read()
file.close()

chat_words_map_dict_en = {}
chat_words_list_en = []
for line in chat_words_str_en.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list_en.append(cw)
        chat_words_map_dict_en[cw] = GoogleTranslator(source='english', target='portuguese').translate(cw_expanded.lower())
        # chat_words_map_dict_en[cw] = translate_text_pt(cw_expanded.lower())
chat_words_list_en = set(chat_words_list_en)
# chat_words_map_dict_en

In [37]:
def chat_words_conversion_en(text):
    if text is None:
        return None
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list_en:
            new_text.append(chat_words_map_dict_en[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

text = "One minute BRB"
print('\033[1mAntes:\033[0m', text)
print('\033[1mDepois:\033[0m', chat_words_conversion_en(text.lower()))

In [38]:
# Leitura do dataset das abreviaturas em português
chat_words_str_pt = pd.read_excel('Datasets_Vodafone/Auxiliares/Abreviaturas_Portugues.xlsx')
chat_words_str_pt

In [39]:
# Criar um dicionário para mapear as abreviações e seus significados
chat_words_map_dict_pt = dict(zip(chat_words_str_pt['Abreviatura'], chat_words_str_pt['Palavra']))

# Criar uma lista de abreviações
chat_words_list_pt = set(chat_words_str_pt['Abreviatura'])

def chat_words_conversion_pt(text):
    if text is None:
        return None
    new_text = []
    for w in text.split():
        if w.lower() in chat_words_list_pt:
            new_text.append(chat_words_map_dict_pt[w.lower()])
        else:
            new_text.append(w)
    return " ".join(new_text)

text = "mn blz tm pdc gnt"
print('\033[1mAntes:\033[0m', text)
print('\033[1mDepois:\033[0m', chat_words_conversion_pt(text.lower()))

In [40]:
# Aplicar aos datasets
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: chat_words_conversion_en(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: chat_words_conversion_en(text))

Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: chat_words_conversion_pt(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: chat_words_conversion_pt(text))

In [41]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

# <a class='anchor' id='99999'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: transparent; 
            padding: 10px; color: white; border-radius: 300px; text-align: center;
            border: 2px solid #A30000;">
    <center><h2 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: #A30000;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>Pré-Processamento | Principais Etapas</b></h2></center>
</div>

* *Tokenization*         ✅
* *Removal of StopWords* ✅
* *Stemming*             ✅
* *Lemmatization*        ✅
* *POS Tagging*          ✅
* *Naming Entities*      ✅
* *Desambiguação*        ✅
* *N-Gramas*             ✅

In [42]:
## # Frase exemplo:
text = 'As operadoras de telecomunicações oferecem uma gama de serviços de comunicação.'
print("\033[1mAntes:\033[0m", text)
text = text.lower()
text = remove_urls(text)
text = remove_punctuation(text)
text = remove_extra_spaces(text)
text = chat_words_conversion_en(text)
text = chat_words_conversion_pt(text)
print("\033[1mDepois:\033[0m", text)

### Removal of *StopWords*

As stopwords são palavras que ocorrem habitualmente numa língua, como **"o"**, **"a"**, etc. Na maioria das vezes, podem ser removidas do texto, uma vez que não fornecem informações valiosas para a análise a jusante. Em casos como a etiquetagem da Parte do Discurso, não se deve remover, pois fornecem informações muito valiosas sobre o `POS`.

Estas listas de palavras já estão compiladas para diferentes línguas no pacote `nltk`.

In [43]:
# import nltk
# nltk.download('stopwords')

In [44]:
from nltk.corpus import stopwords

# Obtenha a lista padrão de stopwords em português
STOPWORDS = set(stopwords.words('portuguese'))

# Remova a palavra "nos" da lista de stopwords
STOPWORDS.discard("nos")

def remove_stopwords(text):
    """Função personalizada para remover as stopwords"""
    return " ".join([word for word in str(text).split() if word.lower() not in STOPWORDS])

# Teste da função com um exemplo de texto
# texto = "O gato preto atravessou a rua nos à noite. O cachorro latiu. Nos comentários, as pessoas elogiaram a NOS."
print("\033[1mAntes:\033[0m", text)
print("\033[1mDepois:\033[0m", remove_stopwords(text))

In [45]:
# Aplicar ao dataset
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: remove_stopwords(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: remove_stopwords(text))

In [46]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

### Stemming

O stemming é o processo de redução de palavras flexionadas (ou por vezes derivadas) ao seu radical, base ou raiz (de [Wikipedia](https://en.wikipedia.org/wiki/Stemming))

Por exemplo, se existirem duas palavras no corpus `walks` e `walking`, então o stemming reduzirá o sufixo para as tornar `walk`. Mas, por exemplo, se houver duas palavras `console` e `consoling`, o stemmer irá remover o sufixo e torná-las `consol`, o que não é uma palavra correcta em inglês.

Existem vários tipos de algoritmos de ***stemming*** disponíveis e um dos mais famosos é o porter stemmer que é muito utilizado. Pode-se usar o pacote nltk para o mesmo.

In [47]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

print("\033[1mAntes:\033[0m", text)
print("\033[1mDepois:\033[0m", stem_words(text))

In [48]:
# Aplicar ao dataset
Facebook_Posts['post_text_clean'] = Facebook_Posts['post_text_clean'].swifter.apply(lambda text: remove_stopwords(text))
Facebook_Comments['comment_text_clean'] = Facebook_Comments['comment_text_clean'].swifter.apply(lambda text: remove_stopwords(text))

In [49]:
Facebook_Posts[['post_text','post_text_clean']]
Facebook_Comments[['comment_text','comment_text_clean']]

---

### Lemmatization

Lematização é o processo de agrupar inflexões de uma palavra numa forma base ou de recuperar a forma base da mesma palavra, agrupando diferentes formas de conjugar um verbo, por exemplo, num só formato, segue-se um exemplo:

In [50]:
# nltk.download('omw-1.4')
# !pip install spacy
# !python -m spacy download pt_core_news_sm

In [51]:
# Carregar o modelo em português
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(text):
    # Processar o texto com o modelo spaCy
    doc = nlp(text)
    # Lematizar cada token no texto e retornar o texto lematizado
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

# texto = "Felizes são aqueles que sonham."
print("\033[1mAntes:\033[0m", text)
print("\033[1mDepois:\033[0m", lemmatize_text(text))

### POS (Part-Of-Speech) Tagging

O processo de tagging tem como principal obejtivo atribuir e identificar uma classe lexical a cada palavra do corpo do texto, de forma a ter muitas palavras relacionadas e formas de palavras numa única forma canónica se possível (não necessariamente a forma básica).

In [52]:
nlp = spacy.load("pt_core_news_sm")
doc = nlp(text)
print("\033[1mAntes:\033[0m", doc.text)
print("\033[1mDepois:\033[0m")
for token in doc:
    print(token.text, token.pos_, token.dep_)

> Para a análise pretendida **não é relevante aplicar esta técnica** aos *datasets*

## Tokenization 

A tokenização, no domínio do *Processamento de Linguagem Natural* (NLP), refere-se ao processo de conversão de uma sequência de texto em partes mais pequenas, conhecidas como tokens. 

Estes **`tokens`** podem ser tão pequenos como caracteres ou tão longos como palavras. A principal razão pela qual este processo é importante é o facto de ajudar os algoritmos a compreender a linguagem humana, dividindo-a em partes pequenas, que são mais fáceis de analisar.

In [53]:
# ======================== Pré-processamento com vista à representação de documentos ========================
# Como ler o Léxico de sentimentos ?
# R: Os dataframes e as séries são pouco eficientes. Logo, uma melhor solução será 
#    representar tudo com um dicionário [.to_dict()], o que tornará o nosso código muito mais rápido...

# Tokenização com NLTK [1º Alterantiva]
tokens = nltk.word_tokenize(text)
print(tokens)

In [54]:
from nltk.tokenize import TweetTokenizer

# Tokenização com NLTK [2º Alterantiva] - Identifica smiles ':)' e '#s'
mytkzr = TweetTokenizer()
print(mytkzr.tokenize(text))

In [55]:
# Depois do pré-processamento inicial, calcular a frequência de cada token
freq=collections.Counter(tokens)
print(freq)

---

## Naming Entities

O processo de nomear entidades envolve classificar nomes e outras entidades em categorias específicas, como organizações, pessoas, locais, expressões temporais (como tempo e data) e quantidades (valores monetários, percentagens e números de telefone). 

Uma das abordagens utiliza Learning Systems, os quais empregam técnicas estatísticas ou de machine learning para descobrir regras e critérios automaticamente, geralmente exigindo grandes volumes de dados para o treinamento dos modelos.

In [56]:
# Carregar o modelo em português
nlp = spacy.load("pt_core_news_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

texto = "A VODAFONE, a NOS e a MEO são as maiores operadoras de telecomunicação em Portugal."
print("\033[1mTexto:\033[0m", texto, '\n')
print("\033[1mEntidades identificadas:\033[0m")
for entity, label in extract_entities(texto):
    print(f"\033[1mEntidade:\033[0m {entity}, \033[1mTipo:\033[0m {label}")

> Devido ao tipo de estratégia que vamos optar no ***Modeling*** não é necessário aplicar ao *dataset*

---

## Desambiguação

Este é o processo de remover a ambiguidade de um texto, atribuindo um significado específico a uma palavra entre várias possibilidades existentes. Geralmente, o sentido é determinado por um dicionário, mas em casos de polissemia ou homonímia, a escolha deve ser baseada no contexto. 

Numa perspectiva de extração automática, selecionar a interpretação correta pode ser desafiador quando há apenas um token. É importante notar que uma palavra pode ter múltiplos ou até contraditórios significados, como "título" ou "apple". A desambiguação é essencial para garantir a compreensão correta do texto.

In [57]:
# nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn
# wn.langs() # 'por' para Português

# Função para desambiguar uma palavra com base no contexto
def desambiguar_palavra(palavra, contexto):
    synsets_palavra = wn.synsets(palavra, lang='por')
    if not synsets_palavra:
        return palavra  # Se não houver synsets, retornar a própria palavra
    else:
        melhor_synset = None
        melhor_similaridade = -1
        for syn_palavra in synsets_palavra:
            for token in nltk.word_tokenize(contexto):
                synsets_token = wn.synsets(token, lang='por')
                for syn_token in synsets_token:
                    similaridade = syn_palavra.path_similarity(syn_token)
                    if similaridade and similaridade > melhor_similaridade:
                        melhor_synset = syn_palavra
                        melhor_similaridade = similaridade
        if melhor_synset:
            return melhor_synset.definition()
        else:
            return palavra

In [58]:
# Exemplo de uso
tokens = nltk.word_tokenize(texto)
print("\033[1mTexto:\033[0m", texto, '\n')
for token in tokens:
    significado = desambiguar_palavra(token, texto)
    print(f"\033[1m{token}:\033[0m {significado}")

---

## 🔂 Unigramas

In [59]:
from nltk.tokenize import word_tokenize

# Texto de exemplo
# texto = "O gato preto atravessou a rua à noite. O gato bebeu leite."
print("\033[1mAntes:\033[0m", text)
# Tokenização do texto
tokens = word_tokenize(text)

# Exibição dos unigramas
print("\033[1mUnigramas:\033[0m")
for unigrama in tokens:
    print(unigrama)

In [60]:
#!pip install Wordcloud

In [61]:
# Texto de exemplo
# texto = "O gato preto atravessou a rua à noite. O gato bebeu leite."
print("\033[1mAntes:\033[0m", text, '\n')
# Tokenização do texto
tokens = word_tokenize(text)

# Contagem das palavras
contagem_palavras = Counter(tokens)

# Exibir as palavras mais frequentes
print("\033[1mPalavras mais frequentes:\033[0m")
for palavra, frequencia in contagem_palavras.most_common(len(contagem_palavras)):  # As 5 palavras mais frequentes
    print(f"\033[1m{palavra}:\033[0m {frequencia} vezes")

In [62]:
contagem_palavras

---

### 💬 Aplicação à base de dados já limpa para os *posts*

In [63]:
# Tokenização do texto dos posts
tokens = []
for post in Facebook_Posts['post_text_clean']:
    tokens.extend(word_tokenize(str(post)))

# Contagem das palavras
contagem_palavras_post_text = Counter(tokens)

# # Exibir as palavras mais frequentes
# print("\033[1mPalavras mais frequentes:\033[0m")
# for palavra, frequencia in contagem_palavras_post_text.most_common(50):  # As 50 palavras mais frequentes
#     print(f"{palavra}: {frequencia} vezes")

In [64]:
# import csv

# # Criação do arquivo CSV - Unigrama Post
# with open('Datasets_Vodafone/TextMining/Unigramas_Post.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Token", "Frequência"])

#     # Escrita dos tokens e suas frequências no arquivo CSV
#     for token, frequencia in contagem_palavras_post_text.items():
#         escritor.writerow([token, frequencia])

In [65]:
print("\033[1mNº de Tokens/Unigramas escritos na variável post_text:\033[0m",sum(contagem_palavras_post_text.values()), "\033[1mdos quais\033[0m", 
      len(contagem_palavras_post_text), "\033[1msão únicos\033[0m")

In [66]:
# !pip install wordcloud

In [67]:
from wordcloud import WordCloud

# Define a function to plot word cloud
def plot_cloud(wordcloud):
    fig = plt.figure(figsize=(25, 17), dpi=80)
    plt.tight_layout(pad=0)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.box(False)
    plt.show()
    plt.close() 

In [68]:
# Criar um objeto WordCloud
wordcloud = WordCloud(width=800, height=400, background_color="white", colormap='OrRd') \
            .generate_from_frequencies(contagem_palavras_post_text)

# Plotar a nuvem de palavras
plot_cloud(wordcloud)

### 💬 Aplicação à base de dados já limpa para os *comments*

In [69]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Tokenização do texto dos posts
tokens = []
for post in Facebook_Comments['comment_text_clean']:
    tokens.extend(word_tokenize(str(post)))

# Contagem das palavras
contagem_palavras_comment_text = Counter(tokens)

# # Exibir as palavras mais frequentes
# print("\033[1mPalavras mais frequentes:\033[0m")
# for palavra, frequencia in contagem_palavras_comment_text.most_common(50):  # As 50 palavras mais frequentes
#     print(f"{palavra}: {frequencia} vezes")

In [70]:
# import csv

# # Criação do arquivo CSV - Unigrama Comment
# with open('Datasets_Vodafone/TextMining/Unigramas_Comment.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Token", "Frequência"])

#     # Escrita dos tokens e suas frequências no arquivo CSV
#     for token, frequencia in contagem_palavras_comment_text.items():
#         escritor.writerow([token, frequencia])

In [71]:
print("\033[1mNº de Tokens/Unigramas escritos na variável comment_text:\033[0m",sum(contagem_palavras_comment_text.values()), "\033[1mdos quais\033[0m", 
      len(contagem_palavras_comment_text), "\033[1msão únicos\033[0m")

---
Entre as operadoras, pode-se observar que a mais mencionada nos comentários é a **NOS**, em oitavo lugar das palavras mais frequentes, seguida pela **MEO**, em décimo sétimo lugar, já a **Vodafone** encontra-se em quadragésiomo nono lugar. 

É de notar que na limpeza prévia não considerou o sentido de cada comentário onde a palavra "nos" era utilizada, daí poder não estar completamente preciso em termos de ser a operadora NOS (e não o "nos" a nível gramatical). 

---

In [72]:
# Criar um objeto WordCloud
wordcloud = WordCloud(width=800, height=400, background_color="white", colormap='OrRd') \
            .generate_from_frequencies(contagem_palavras_comment_text)

# Plotar a nuvem de palavras
plot_cloud(wordcloud)

## 🔗 Bigramas e Trigramas

In [73]:
# NGram function - Fonte: https://www.kaggle.com/code/bwandowando/generate-bigram-and-trigram-wordcloud
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n=2):
    text = str(text)
    n_grams = ngrams(text.split(), n)
    returnVal = []
    
    try:
        for grams in n_grams:
            returnVal.append('_'.join(grams))
    except(RuntimeError):
        pass
        
    return ' '.join(returnVal).strip()

In [74]:
from nltk.tokenize import word_tokenize
from nltk.util import bigrams

# Texto de exemplo
texto = "A Vodafone tentou comprar a NOS e a MEO mas falhou de novo."

# Tokenização do texto
tokens = word_tokenize(texto)

# Criação de bi-grafos
bi_grafos = list(bigrams(tokens))

# Exibição dos bi-grafos
print("\033[1mBi-grafos:\033[0m")
for bi_grafo in bi_grafos:
    print(bi_grafo)

> Consegue-se perceber como os bigramas funcionam

### Aplicação dos `Bigramas` e `Trigramas` à base de dados limpa para os *posts*

In [75]:
# Bigramas 'post_text'
bi_post_text = Facebook_Posts['post_text_clean'].swifter.apply(get_ngrams, n=2)

bi_post_text_string_list = bi_post_text.tolist()
bi_post_text_string = ' '.join(bi_post_text_string_list)

In [76]:
# # Criação do arquivo CSV - Bigramas Posts
# with open('Datasets_Vodafone/TextMining/Bigramas_Post.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Bigrama", "Frequência"])

#     # Contagem dos bigramas
#     contagem_bigramas = Counter(bi_post_text_string.split())

#     # Escrita dos bigramas e suas frequências no arquivo CSV
#     for bigrama, frequencia in contagem_bigramas.items():
#         escritor.writerow([bigrama, frequencia])

In [77]:
# Contar bigramas
bigramas_post_text_str = Counter(bi_post_text_string.split())

print("\033[1mNº de Bigramas escritos na variável post_text:\033[0m",sum(bigramas_post_text_str.values()), 
      "\033[1mdos quais\033[0m", len(bigramas_post_text_str), "\033[1msão únicos\033[0m")

In [78]:
wordcloud = WordCloud(width = 7680, height = 4320, random_state=1, 
                      background_color='white', colormap='OrRd', # max_words = 75, 
                      collocations=False, normalize_plurals=False).generate(bi_post_text_string)

# Plot
plot_cloud(wordcloud)

In [79]:
# Trigramas 'post_text'
tri_post_text = Facebook_Posts['post_text_clean'].swifter.apply(get_ngrams, n=3)
tri_post_text_string_list = tri_post_text.tolist()
tri_post_text_string = ' '.join(tri_post_text_string_list)

In [80]:
# # Criação do arquivo CSV - Trigramas Posts
# with open('Datasets_Vodafone/TextMining/Trigramas_Post.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Trigrama", "Frequência"])

#     # Contagem dos trigramas
#     contagem_trigramas = Counter(tri_post_text_string.split())

#     # Escrita dos trigramas e suas frequências no arquivo CSV
#     for trigrama, frequencia in contagem_trigramas.items():
#         escritor.writerow([trigrama, frequencia])

In [81]:
# Contar trigramas
trigramas_post_text_str = Counter(tri_post_text_string.split())

print("\033[1mNº de Trigramas escritos na variável post_text:\033[0m",sum(trigramas_post_text_str.values()), 
      "\033[1mdos quais\033[0m", len(trigramas_post_text_str), "\033[1msão únicos\033[0m")

In [82]:
wordcloud = WordCloud(width = 7680, height = 4320, random_state=1, 
                      background_color='white', colormap='OrRd', # max_words = 75, 
                      collocations=False, normalize_plurals=False).generate(tri_post_text_string)

# Plot
plot_cloud(wordcloud)

### Aplicação dos Bigramas e Trigramas para a base de dados dos *comments* limpa

In [83]:
# Bigramas 'comment_text'
bi_comment_text = Facebook_Comments['comment_text_clean'].swifter.apply(get_ngrams, n=2)

bi_comment_text_string_list = bi_comment_text.tolist()
bi_comment_text_string = ' '.join(bi_comment_text_string_list)

# Contar bigramas
bigramas_comment_text_str = Counter(bi_comment_text_string.split())

print("\033[1mNº de Bigramas escritos na variável comment_text:\033[0m",sum(bigramas_comment_text_str.values()), 
      "\033[1mdos quais\033[0m", len(bigramas_comment_text_str), "\033[1msão únicos\033[0m")

In [84]:
# # Criação do arquivo CSV - Bigramas Comment
# with open('Datasets_Vodafone/TextMining/Bigramas_Comment.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Bigramas", "Frequência"])

#     # Escrita dos bigramas e suas frequências no arquivo CSV
#     for bigrama, frequencia in bigramas_comment_text_str.items():
#         escritor.writerow([bigrama, frequencia])

In [85]:
wordcloud = WordCloud(width = 7680, height = 4320, random_state=1, 
                      background_color='white', colormap='OrRd', # max_words = 75, 
                      collocations=False, normalize_plurals=False).generate(bi_comment_text_string)

# Plot
plot_cloud(wordcloud)

In [86]:
# Trigramas 'comment_text'
tri_comment_text = Facebook_Comments['comment_text_clean'].swifter.apply(get_ngrams, n=3)
tri_comment_text_string_list = tri_comment_text.tolist()
tri_comment_text_string = ' '.join(tri_comment_text_string_list)

# Contar trigramas
trigramas_comment_text_str = Counter(tri_comment_text_string.split())

print("\033[1mNº de Bigramas escritos na variável comment_text:\033[0m",sum(trigramas_comment_text_str.values()), 
      "\033[1mdos quais\033[0m", len(trigramas_comment_text_str), "\033[1msão únicos\033[0m")

In [87]:
# # Criação do arquivo CSV - Trigramas Comment
# with open('Datasets_Vodafone/TextMining/Trigramas_Comment.csv', 'w', newline='', encoding='utf-8') as arquivo:
#     escritor = csv.writer(arquivo)
#     escritor.writerow(["Trigramas", "Frequência"])

#     # Escrita dos trigramas e suas frequências no arquivo CSV
#     for trigrama, frequencia in trigramas_comment_text_str.items():
#         escritor.writerow([trigrama, frequencia])

In [88]:
wordcloud = WordCloud(width = 7680, height = 4320, random_state=1, 
                      background_color='white', colormap='OrRd', # max_words = 75, 
                      collocations=False, normalize_plurals=False).generate(tri_comment_text_string)

# Plot
plot_cloud(wordcloud)

---

### Alternativa com o `Likelihood Ratio`

In [135]:
# from nltk.tokenize import word_tokenize
# from nltk.collocations import BigramCollocationFinder
# from nltk.metrics import BigramAssocMeasures

# # Tokenização dos textos dos posts
# tokens = []
# for post in Facebook_Posts['post_text_clean']:
#     tokens.extend(word_tokenize(str(post)))

# # Gerar bigramas ---- Alternativa
# finder = BigramCollocationFinder.from_words(tokens)
# bigramas = finder.nbest(BigramAssocMeasures.likelihood_ratio, 100)  # Escolha dos 100 melhores bigramas

# # Exibir os bigramas
# # print("Bigramas mais frequentes:")
# # for bigrama in bigramas:
# #     print(bigrama)

In [126]:
# bigramas_freq

In [136]:
# # Converter os bigramas em uma única string
# bigramas_str = ' '.join(['_'.join(bigram) for bigram in bigramas])

# # Criar um objeto WordCloud
# wordcloud = WordCloud(width=800, height=400, background_color="white", colormap='OrRd').generate(bigramas_str)

# # Plotar a nuvem de palavras
# plot_cloud(wordcloud)

> Consegue-se ter uma melhor ideia sobre o que é que se refere a palavra "nos"

In [143]:
# from nltk.tokenize import word_tokenize
# from nltk.collocations import BigramCollocationFinder
# from nltk.metrics import BigramAssocMeasures

# # Tokenização dos textos dos comments
# tokens = []
# for post in Facebook_Comments['comment_text_clean']:
#     tokens.extend(word_tokenize(str(post)))

# # Gerar bigramas
# finder = BigramCollocationFinder.from_words(tokens)
# bigramas = finder.nbest(BigramAssocMeasures.likelihood_ratio, 100)  # Escolha dos 50 melhores bigramas

# Exibir os bigramas
# print("Bigramas mais frequentes:")
# for bigrama in bigramas:
#     print(bigrama)

In [142]:
# Converter os bigramas em uma única string
# bigramas_str = ' '.join(['_'.join(bigram) for bigram in bigramas])

# Criar um objeto WordCloud
# wordcloud = WordCloud(width=800, height=400, background_color="white", colormap='OrRd').generate(bigramas_str)

# Plotar a nuvem de palavras
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

---

## 🗃️ Integração de Dados

In [59]:
# Compilar as bases de dados
Facebook_Posts_Comments = pd.merge(Facebook_Posts, Facebook_Comments, how = 'right', 
                                   on = list(Facebook_Comments.columns.intersection(Facebook_Posts.columns)))

# Facebook_Comments.columns.intersection(Facebook_Posts.columns) =
#           ['page', 'post_id', 'post_text', 'post_date', 'post_reactions', 'post_comments', 
#            'post_shares', 'post_day', 'post_month', 'post_year', 'post_hour', 'post_link']

Facebook_PCU = pd.merge(Facebook_Posts_Comments, Facebook_Users, how='left', on=['user_link', 'user_name'])

In [60]:
# Reorganizar as colunas
colunas_organizadas = [
    'page', 'post_id', 'post_link', 'post_date', 'post_day', 'post_month', 'post_year', 'post_hour', 
    'post_reactions', 'post_comments', 'post_shares', 'post_text', 'post_text_clean', 'post_language',
    
    'comment_id', 'comment_link', 'comment_date', 'comment_day_ago', 'comment_reactions', 'comment_num_responses',
    'comment_operator_responded', 'comment_text', 'comment_language', 'comment_text_clean',
    
    'user_name', 'user_link', 'user_current_city', 'user_hometown', 
    'user_freguesia', 'user_concelho', 'user_distrito', 'user_pais', 'user_city_not_portugal', 'user_predicted_genre'
]

Facebook_PCU = Facebook_PCU[colunas_organizadas]

```Python
# Guardar os dados integrados em formato .txt
Facebook_PCU.to_csv('Datasets_Vodafone/Facebook_PCU.txt', sep='\t', index=False, encoding='utf-8')
```

---

# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>4 | Modeling</b></h1></center>
</div>

In [20]:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia [Para PCs com NVIDIA]
# !pip install --upgrade accelerate
# ---------
# !pip install --upgrade transformers
# ---------
# !pip install simpletransformers ---------
# !pip install xformers           ---------

In [21]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [22]:
import transformers
print(transformers.__version__)

In [23]:
# Fonte: https://huggingface.co/models
from transformers import pipeline, AutoModel, AutoTokenizer

In [24]:
import torch
torch.cuda.empty_cache()  # You may use this command to clear your cache
torch.cuda.is_available() # You may use this command to check if you have gpu or not

In [25]:
# automatically chose CPU or GPU for inference, depending on your hardware
import torch
print(torch.__version__)
device = torch.cuda.current_device() if torch.cuda.is_available() else -1
# -1 == CPU ; 0 == GPU
print(device)

In [26]:
# Hugging Face
import os
os.environ['HF_HOME'] = 'C:/HuggingFace'               #   -----------> Caso de Erro -> Alterar a paste de '.cache'
os.environ['HF_DATASETS_CACHE'] = 'C:/HuggingFace'
os.environ['TRANSFORMERS_CACHE'] = 'C:/HuggingFace'

In [27]:
# Import da base de dados integrada 
# Facebook_PCU = pd.read_csv('Datasets_Vodafone/Facebook_PCU.txt', sep='\t', encoding='utf-8')

# <a class='anchor' id='4.1'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: transparent; 
            padding: 10px; color: white; border-radius: 300px; text-align: center;
            border: 2px solid #A30000;">
    <center><h2 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: #A30000;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>Sentiment Analysis</b></h2></center>
</div>

----

## 🟡 HuggingFace [Meta-Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B)

In [30]:
# from huggingface_hub import login
# login()

In [31]:
# import transformers

# # Carregar o modelo local
# model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

# pipeline = transformers.pipeline(
#     "text-generation", 
#     model=model, 
#     tokenizer=tokenizer,    
#     model_kwargs={"torch_dtype": torch.bfloat16}, 
#     device_map="auto"
# )

In [33]:
# pipeline("Hey how are you doing today?")

> Requisitos minimos não correspondidos

---

## 🟡 HuggingFace [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)

In [10]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# device = "cuda" # the device to load the model onto

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]

# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

# model_inputs = encodeds.to(device)
# model.to(device)

# generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

> Requisitos minimos não correspondidos

---

## 🟡 HuggingFace [distilbert-base-multilingual-cased-sentiments-student](https://huggingface.co/lxyuan/distilbert-base-multilingual-cased-sentiments-student)

In [34]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

In [35]:
# Teste em Inglês
distilled_student_sentiment_classifier ("I love this movie and i would watch it again and again!")
# >> [[{'label': 'positive', 'score': 0.9731044769287109},
#   {'label': 'neutral', 'score': 0.016910076141357422},
#   {'label': 'negative', 'score': 0.009985478594899178}]]

In [36]:
# Teste em Português
distilled_student_sentiment_classifier("O serviço não é de qualidade")

> Baixa qualidade de classificação -> Não Adaptado para **Português**

---

## 🟡 HuggingFace [bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment)

In [38]:
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [39]:
data = ["A Vodafone é horrível!", "O Rui é tão feio", "O senhor é maluco!", "O mapa está feito!", "Força FCP"]
classifier(data)

In [40]:
comments = Facebook_Comments['comment_text'][0:10].to_list()
output = classifier(comments)
df_result = pd.DataFrame(output)
df_result['comment'] = comments
df_result

> Não se justifica esta subdivisão para a análise de sentimentos!

---

## 🟡 HuggingFace [twitter-xlm-roberta-base-sentiment-finetunned](https://huggingface.co/citizenlab/twitter-xlm-roberta-base-sentiment-finetunned)

In [35]:
model_path = "citizenlab/twitter-xlm-roberta-base-sentiment-finetunned"
TXRBSF_sentiment_classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device=device)

# Defina a função de classificação
def TXRBSF_classify_sentiment(text):
    if text.strip() == '' or text is None:
        return np.NaN, np.NaN
    outputs = TXRBSF_sentiment_classifier(text, batch_size=8)
    return outputs[0]['label'], outputs[0]['score']

In [36]:
# Teste do Modelo

# Exemplo
print("\033[1mElementos da Pipeline:\033[0m \n",
      TXRBSF_sentiment_classifier("A Vodafone tem um serviço de baixa qualidade!"))

print("\n\033[1mOutput da Função TXRBSF_classify_sentiment:\033[0m \n",
      TXRBSF_classify_sentiment("A Vodafone tem um serviço de baixa qualidade!"))

```Python
# ------------------------------ Já corri - 30.04.2024 ------------------------------
# Aplicar o twitter-xlm-roberta-base-sentiment-finetunned ao dataset dos posts
TXRBSF_classify_posts_sentiments = Facebook_Posts['post_text'].progress_apply(lambda x: TXRBSF_classify_sentiment(x))
labels, scores = zip(*[(label, score) for label, score in TXRBSF_classify_posts_sentiments])

Facebook_Posts['TXRBSF_post_sentiment_label'] = labels
Facebook_Posts['TXRBSF_post_sentiment_score'] = scores

# Salvar a nova coluna como um arquivo CSV
Facebook_Posts[['post_id','post_text', 'TXRBSF_post_sentiment_label', 'TXRBSF_post_sentiment_score']].to_csv('Datasets_Vodafone/TextMining/TXRBSF_post_sentiment.csv', index=False, encoding='utf-8')
```

In [61]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
TXRBSF_post_sentiment = pd.read_csv('Datasets_Vodafone/TextMining/TXRBSF_post_sentiment.csv') \
                          .drop(['post_id','post_text'], axis=1)
Facebook_Posts = Facebook_Posts.assign(**TXRBSF_post_sentiment)

In [62]:
Facebook_Posts[['post_text', 'TXRBSF_post_sentiment_label', 'TXRBSF_post_sentiment_score']].sample(5)

In [164]:
# Tabela de Frequências de sentimentos
print("\033[1mNº de NAs:\033[0m", Facebook_Posts['TXRBSF_post_sentiment_label'].isna().sum())
pd.DataFrame({
    'n':Facebook_Posts['TXRBSF_post_sentiment_label'].value_counts(),
    '%':round(Facebook_Posts['TXRBSF_post_sentiment_label'].value_counts(normalize=True) * 100,1)})

> O nº de **`NAs`** bate certo com os *posts* sem texto!

---

```Python
# ------------------------------ Já corri - 01.05.2024 ------------------------------
# Aplicar o twitter-xlm-roberta-base-sentiment-finetunned ao dataset dos comentários
TXRBSF_classify_comments_sentiments = Facebook_Comments['comment_text'].progress_apply(lambda x: TXRBSF_classify_sentiment(x))
labels, scores = zip(*[(label, score) for label, score in TXRBSF_classify_comments_sentiments])

Facebook_Comments['TXRBSF_comment_sentiment_label'] = labels
Facebook_Comments['TXRBSF_comment_sentiment_score'] = scores

# Salvar a nova coluna como um arquivo CSV
Facebook_Comments[['comment_id', 'comment_text', 'TXRBSF_comment_sentiment_label', 'TXRBSF_comment_sentiment_score']] \
    .to_csv('Datasets_Vodafone/TextMining/TXRBSF_comment_sentiment.csv', index=False, encoding='utf-8')
```

In [64]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
TXRBSF_comment_sentiment = pd.read_csv('Datasets_Vodafone/TextMining/TXRBSF_comment_sentiment.csv') \
                             .drop(['comment_id','comment_text'], axis=1)
Facebook_Comments = Facebook_Comments.assign(**TXRBSF_comment_sentiment)

In [65]:
# Verificar as 4 observações sem 'TXRBSF_comment_sentiment_label' e 'TXRBSF_comment_sentiment_score' -> Não têm texto
# Facebook_Comments[Facebook_Comments[['TXRBSF_comment_sentiment_label', 'TXRBSF_comment_sentiment_score']].isna().any(axis=1)]

In [66]:
Facebook_Comments[['comment_text', 'TXRBSF_comment_sentiment_label', 'TXRBSF_comment_sentiment_score']].sample(5)

In [170]:
TXRBSF_comment_sentiment_label_df = pd.DataFrame({
    'n':Facebook_Comments['TXRBSF_comment_sentiment_label'].value_counts(),
    '%':round(Facebook_Comments['TXRBSF_comment_sentiment_label'].value_counts(normalize=True) * 100,1)})

TXRBSF_comment_sentiment_label_df.index.name = 'TXRBSF Model'
TXRBSF_comment_sentiment_label_df

---

## 🟡 HuggingFace [mDeBERTa-v3-base-xnli-multilingual-nli-2mil7](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7)

In [43]:
# Defina o pipeline de classificação
model_path = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
moritz_classifier = pipeline("zero-shot-classification", model=model_path, tokenizer=model_path, device=device)

# Nota: para utlizar este modelo temos de dar moritz_classifier(sequence_to_classify, candidate_labels, multi_label=False)

# Defina as classes a classificar pelo modelo
candidate_labels = ['Positive', 'Neutral', 'Negative']

# Defina a função de classificação
def mDeBERTa_classify_sentiment(texts):
    outputs = moritz_classifier([texts], candidate_labels, multi_label=False, batch_size=8)
    labels = [output['labels'][0] for output in outputs]
    scores = [output['scores'][0] for output in outputs]
    return labels, scores

In [44]:
# Exemplo
print("\033[1mElementos da Pipeline:\033[0m \n",
      moritz_classifier("A Vodafone tem um serviço de baixa qualidade!", candidate_labels, multi_label=False))

print("\n\033[1mOutput da Função mDeBERTa_classify_sentiment:\033[0m \n",
      mDeBERTa_classify_sentiment("A Vodafone tem um serviço de baixa qualidade!"))

```Python
# ------------------------------ Já corri - 30.04.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos posts
mDeBERTa_posts_sentiments = Facebook_Posts['post_text'].progress_apply(lambda x: mDeBERTa_classify_sentiment([x]))
labels, scores = zip(*[(label, score) for label, score in mDeBERTa_posts_sentiments])

Facebook_Posts['mDeBERTa_post_sentiment_label'] = labels
Facebook_Posts['mDeBERTa_post_sentiment_score'] = scores

# Salvar a nova coluna como um arquivo CSV
Facebook_Posts[['post_id', 'post_text', 'mDeBERTa_post_sentiment_label', 'mDeBERTa_post_sentiment_score']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_sentiment.csv', encoding='utf-8', index=False)
```

In [68]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_post_sentiment = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_sentiment.csv', encoding='utf-8')  \
                          .drop(['post_id','post_text'], axis=1)
Facebook_Posts = Facebook_Posts.assign(**mDeBERTa_post_sentiment)

# Limpar a variável 'mDeBERTa_post_sentiment_label'
Facebook_Posts['mDeBERTa_post_sentiment_label'] = Facebook_Posts['mDeBERTa_post_sentiment_label'].str.strip("['']")

# Converter a variável 'mDeBERTa_post_sentiment_score' para float
Facebook_Posts['mDeBERTa_post_sentiment_score'] = Facebook_Posts['mDeBERTa_post_sentiment_score'].str.strip("['']").astype(float)

In [173]:
mDeBERTa_post_sentiment_label_df = pd.DataFrame({
    'n':Facebook_Posts['mDeBERTa_post_sentiment_label'].value_counts(),
    '%':round(Facebook_Posts['mDeBERTa_post_sentiment_label'].value_counts(normalize=True) * 100,1)
})

mDeBERTa_post_sentiment_label_df.index.name = 'mDeBERTa Model'
mDeBERTa_post_sentiment_label_df

In [70]:
Facebook_Posts[['post_text', 'mDeBERTa_post_sentiment_label', 'mDeBERTa_post_sentiment_score']].sample(5)

---

```Python
# ------------------------------ Já corri - 01.05.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos comentários
mDeBERTa_comments_sentiments = Facebook_Comments['comment_text'].progress_apply(lambda x: mDeBERTa_classify_sentiment([x]))
labels, scores = zip(*[(label, score) for label, score in mDeBERTa_comments_sentiments])

Facebook_Comments['mDeBERTa_comment_sentiment_label'] = labels
Facebook_Comments['mDeBERTa_comment_sentiment_score'] = scores

# Salvar a nova coluna como um arquivo CSV
Facebook_Comments[['comment_id', 'comment_text', 'mDeBERTa_comment_sentiment_label', 'mDeBERTa_comment_sentiment_score']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_sentiment.csv', encoding='utf-8', index=False)
```

In [71]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_comment_sentiment = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_sentiment.csv', encoding='utf-8') \
                             .drop(['comment_id','comment_text'], axis=1)
Facebook_Comments = Facebook_Comments.assign(**mDeBERTa_comment_sentiment)

# Limpar a variável 'mDeBERTa_comment_sentiment_label'
Facebook_Comments['mDeBERTa_comment_sentiment_label'] = Facebook_Comments['mDeBERTa_comment_sentiment_label'].str.strip("['']")

# Converter a variável 'mDeBERTa_comment_sentiment_score' para float
Facebook_Comments['mDeBERTa_comment_sentiment_score'] = Facebook_Comments['mDeBERTa_comment_sentiment_score'].str.strip("['']").astype(float)

In [172]:
mDeBERTa_comment_sentiment_label_df = pd.DataFrame({
    'n':Facebook_Comments['mDeBERTa_comment_sentiment_label'].value_counts(),
    '%':round(Facebook_Comments['mDeBERTa_comment_sentiment_label'].value_counts(normalize=True) * 100,1)})

mDeBERTa_comment_sentiment_label_df.index.name = 'mDeBERTa Model'
mDeBERTa_comment_sentiment_label_df

In [73]:
Facebook_Comments[['comment_text', 'mDeBERTa_comment_sentiment_label', 'mDeBERTa_comment_sentiment_score']].sample(5)

---

## ➕ Juntar Modelos dos Sentimentos

#### `Posts`

In [74]:
from sklearn.metrics import confusion_matrix

# Remova as linhas com valores NaN de ambas as colunas
Facebook_Posts_clean = Facebook_Posts.dropna(subset=['TXRBSF_post_sentiment_label', 'mDeBERTa_post_sentiment_label'])

# Crie a matriz de classificação
conf_matrix = confusion_matrix(Facebook_Posts_clean['TXRBSF_post_sentiment_label'], 
                               Facebook_Posts_clean['mDeBERTa_post_sentiment_label'], 
                               labels=["Positive", "Neutral", "Negative"])

# Crie um DataFrame a partir da matriz de classificação
conf_matrix_df = pd.DataFrame(conf_matrix,
                              index=["Positive", "Neutral", "Negative"],
                              columns=["Positive", "Neutral", "Negative"])
conf_matrix_df.index.name = "Modelo TXRBSF"
conf_matrix_df.columns.name = 'Modelo mDeBERTa'

display_side_by_side(conf_matrix_df,
                     super_title="Matriz de Classificação dos Resultados de Ambos os Modelos<br></br>",
                     titles=[""])

In [75]:
# Variável dos Sentimentos com os 2 Modelos
# Defina as condições para cada caso
conditions = [
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Positive') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Positive'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Positive') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Neutral'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Positive') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Negative'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Neutral') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Positive'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Neutral') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Neutral'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Neutral') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Negative'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Negative') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Positive'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Negative') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Neutral'),
    (Facebook_Posts['TXRBSF_post_sentiment_label'] == 'Negative') & (Facebook_Posts['mDeBERTa_post_sentiment_label'] == 'Negative')
]

# Defina os valores para cada caso
values = ['Positivo', 'Tendência Positiva', 'Neutro', 'Tendência Positiva', 'Neutro', 'Tendência Negativa', 'Neutro', 'Tendência Negativa', 'Negativo']

# Aplique as condições e os valores à nova coluna
Facebook_Posts['post_sentiment_label'] = np.select(conditions, values)

# Atribua o valor NaN à variável 'post_sentiment_label' nas linhas em que uma das colunas é NaN
Facebook_Posts['post_sentiment_label'] = np.where((Facebook_Posts['TXRBSF_post_sentiment_label'].isna()) | 
                                                           (Facebook_Posts['mDeBERTa_post_sentiment_label'].isna()), 
                                                           np.nan, 
                                                           Facebook_Posts['post_sentiment_label'])

In [76]:
pd.DataFrame(Facebook_Posts['post_sentiment_label'].value_counts())

---

#### `Comments` 

In [77]:
from sklearn.metrics import confusion_matrix

# Remova as linhas com valores NaN de ambas as colunas
Facebook_Comments_clean = Facebook_Comments.dropna(subset=['TXRBSF_comment_sentiment_label', 'mDeBERTa_comment_sentiment_label'])

# Crie a matriz de classificação
conf_matrix = confusion_matrix(Facebook_Comments_clean['TXRBSF_comment_sentiment_label'],
                               Facebook_Comments_clean['mDeBERTa_comment_sentiment_label'],
                               labels=["Positive", "Neutral", "Negative"])

# Crie um DataFrame a partir da matriz de classificação
conf_matrix_df = pd.DataFrame(conf_matrix,
                              index=["Positive", "Neutral", "Negative"],
                              columns=["Positive", "Neutral", "Negative"])
conf_matrix_df.index.name = "Modelo TXRBSF"
conf_matrix_df.columns.name = 'Modelo mDeBERTa'

display_side_by_side(conf_matrix_df,
                     super_title="Matriz de Classificação dos Resultados de Ambos os Modelos<br></br>",
                     titles=[""])

In [78]:
# Variável dos Sentimentos com os 2 Modelos
# Defina as condições para cada caso
conditions = [
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Positive') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Positive'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Positive') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Neutral'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Positive') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Negative'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Neutral') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Positive'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Neutral') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Neutral'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Neutral') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Negative'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Negative') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Positive'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Negative') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Neutral'),
    (Facebook_Comments['TXRBSF_comment_sentiment_label'] == 'Negative') & (Facebook_Comments['mDeBERTa_comment_sentiment_label'] == 'Negative')
]

# Defina os valores para cada caso
values = ['Positivo', 'Tendência Positiva', 'Neutro', 'Tendência Positiva', 'Neutro', 'Tendência Negativa', 'Neutro', 'Tendência Negativa', 'Negativo']

# Aplique as condições e os valores à nova coluna
Facebook_Comments['comment_sentiment_label'] = np.select(conditions, values)

# Atribua o valor NaN à variável 'comment_sentiment_label' nas linhas em que uma das colunas é NaN
Facebook_Comments['comment_sentiment_label'] = np.where((Facebook_Comments['TXRBSF_comment_sentiment_label'].isna()) |
                                                                (Facebook_Comments['mDeBERTa_comment_sentiment_label'].isna()),
                                                                np.nan,
                                                                Facebook_Comments['comment_sentiment_label'])

In [174]:
comment_sentiment_label_df = pd.DataFrame({
    'n':Facebook_Comments['comment_sentiment_label'].value_counts(),
    '%':round(Facebook_Comments['comment_sentiment_label'].value_counts(normalize=True) * 100,1)
})

comment_sentiment_label_df.index.name = None
comment_sentiment_label_df.reindex(['Positivo', 'Tendência Positiva', 'Neutro', 'Tendência Negativa', 'Negativo'])

---

### 🔴 Extra | Variável `post_text_Vodafone/MEO/NOS` e `comment_text_Vodafone/MEO/NOS`

In [80]:
import re

def create_binary_variable(text, names):
    """
    Cria uma variável binária que é 1 se algum dos nomes na lista aparecer no texto,
    e 0 caso contrário.
    """
    # Verifica se os names é do tipo lista ou string
    # Caso seja string, converte numa lista 
    if isinstance(names, str):
        names = [names]
    
    # Para todos os nomes possíveis, verifica a sua existência
    for name in names:
        # Use a função `re.IGNORECASE` para tornar a pesquisa insensível a maiúsculas e minúsculas
        if re.search(r'\b' + re.escape(name) + r'\b', text, re.IGNORECASE):
            return 1
    return 0

In [81]:
# Teste da Função
textos = ["A VoDafone tem bons serviços!", "Vou mudar para a meo e para a nos", "Nós vamos à praia e comando é meu, vodafoneparedesdecoura"]

# Lista de nomes das operadoras
operators = ['Vodafone', 'MEO', 'NOS', 'DIGI']

for texto in textos:
    print("\033[1mTexto:\033[0m", texto)
    for operator in operators:
        print(f"\033[1m    {operator}:\033[0m", create_binary_variable(texto, operator))
    print("\n")

In [82]:
# Lista de Possibilidades de formas como as operadoras podem ser referidas
vodafone_expressoes = ['Vodafone', 'vodafoneparedesdecoura', 'vodafonehboportugal', 'vodafonept', 
                       'supertaçavodafone', 'myvodafonept', 'vodafoneptparticipe', '30anosvodafone', 
                       'vodafonehappy', 'vodafonebusinessconference', 'vodafonerallydeportugal', 'vodafoneem', 
                       'vodafonetv', 'vodafonetvplay', 'minimaratonavodafone', 'vodafonecançãodoano']

meo_expressoes = ['MEO', 'emmeopttvficaemcasa', 'modomeosdoeste', 'meocomo', 'nomeoptligameosurf2022', 'emmeoptmbeachcam', 
                  'meoficaemcasa', 'meosou', 'meosudoeste', 'meomaresvivas', 'emmeoptinstagram', 'meops', 
                  'emmeoptvemcomtudoomeovemcomtudo', 'meosó', 'meouma', 'emmeoptdesporto', 'meofui', 'meobox', 
                  'emmeoptdicaenergia', 'emmeoptprogramameos', 'meoa', 'ligameosurfjorge', 'emmeopt2021bymeo', 
                  'meono', 'meoou', 'meo', 'meoequipamentos', 'meogonão', 'meovisslaproericeira', 'emmeoptmmv24', 
                  'meovergonha', 'meowifi', 'meosmartphones', 'meopor', 'meoentretenimento', 'meohteam', 'meo4o', 
                  'meoestamos', 'meogo', 'meokalorama', 'emmeoptbandeiraverde', 'meose', 'ligameosurf', 'meoos', 'meojá',
                  'festivalmeo', 'emmeoptsonytv4k', 'meonem', 'portugalmeo', 'meoenergia', 'embaixadorameo', 'meoobrigado',
                  'meonao', 'meopodiam', 'ameo', 'meoque', 'meosabe', 'meoproportugal', 'meoserá', 'emmeoptappstv', 
                  'emmeopttv', 'emmeoptsporttvfb', 'emmeoptsporttv', 'modomeosudoeste', 'meodesde', 'meopela', 
                  'emmeoptpacotesgaming', 'emmeopteuro', 'omeovemcomtudo', 'meohá', 'meoo', 'meopara', 'mymeo', 
                  'suportelojaonlinemeopt', 'meo5sentidos', 'tdtmelhorquemeo', 'nomeoptmaisnet', 'meoassim', 'alticemeo', 
                  'meoaltice', 'meo…', 'meocontactei', 'meonão', 'meoé', 'meofiz', 'emmeoptregressoaulas21', 'meohifi', 
                  'emmeopteleven', 'meoe', 'emmeoptblnight', 'emmeoptbluefriday', 'meo5g', 'meocloud', 'emmeoptmeoskyshowtime', 
                  'meomas', 'embaixadormeo', 'emmeopttvcine', 'meos', 'wearemeokalorama', 'emmeoptmvod', 'meopois', 'meopt', 
                  'tdtémelhorqueameo', 'meofeliz', 'campeõesmeo', 'meossão', 'meoparece', 'meoem', 'emmeoptmeovideochat', 
                  'meosem', 'meoripcurlpro', 'emmeoptpassatempos', 'emmeoptbtv', 'meodepois', 'meoinstagram', 'emmeoptmuda21']

nos_expressoes = ['NOS', 'nosvê', 'nos5g', 'palconos', 'nosplay', 'noscompra', 'nospt', 'nosadere', 'nosptnosstudios', 
                  'nosportugal', 'nosptkids', 'nosptadesaosporttvliganos', 'nossabe', 'nosmúsica', 'nos4a2', 
                  'bancadanos', 'nosno', 'liganos', 'nosalive', 'nosptnoskidsapp', 'nãoteprendaspornos', '5gnos', 
                  'instagramcomliganos', 'nosprimaverasound', 'nosptappnoskidsfb', 'nosptnosplay', 'nosnosalive']

digi_expressoes =['DIGI']

In [83]:
# Aplicar a função create_binary_variable aos conjuntos de dados Facebook_Posts e Facebook_Comments
Facebook_Posts['post_text_Vodafone'] = Facebook_Posts['post_text'].progress_apply(lambda x: create_binary_variable(x, vodafone_expressoes))
Facebook_Posts['post_text_MEO'] = Facebook_Posts['post_text'].progress_apply(lambda x: create_binary_variable(x, meo_expressoes))
Facebook_Posts['post_text_NOS'] = Facebook_Posts['post_text'].progress_apply(lambda x: create_binary_variable(x, nos_expressoes))
Facebook_Posts['post_text_DIGI'] = Facebook_Posts['post_text'].progress_apply(lambda x: create_binary_variable(x, digi_expressoes))

Facebook_Comments['comment_text_Vodafone'] = Facebook_Comments['comment_text'].progress_apply(lambda x: create_binary_variable(x, vodafone_expressoes))
Facebook_Comments['comment_text_MEO'] = Facebook_Comments['comment_text'].progress_apply(lambda x: create_binary_variable(x, meo_expressoes))
Facebook_Comments['comment_text_NOS'] = Facebook_Comments['comment_text'].progress_apply(lambda x: create_binary_variable(x, nos_expressoes))
Facebook_Comments['comment_text_DIGI'] = Facebook_Comments['comment_text'].progress_apply(lambda x: create_binary_variable(x, digi_expressoes))

In [84]:
# Função para calcular a %
def calculate_percentage(value, total):
    return round((value / total) * 100,1) if total != 0 else 0

# Tabela de Frequências das váriaveis relativas à referência do nome das operadoras
df_post_text_operatores = pd.DataFrame({
    '0 | Não (n)': Facebook_Posts[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']].apply(lambda x: x.value_counts()[0]),
    '0 | Não (%)': Facebook_Posts[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']].apply(lambda x: calculate_percentage(x.value_counts()[0], len(x))),
    '1 | Sim (n)': Facebook_Posts[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']].apply(lambda x: x.value_counts()[1]),
    '1 | Sim (%)': Facebook_Posts[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']].apply(lambda x: calculate_percentage(x.value_counts()[1], len(x)))
})

df_post_text_operatores.index = [operators]
df_post_text_operatores

In [85]:
# Tabela de Frequências das variáveis relativas à referência do nome das operadoras nos comentários do Facebook
df_comment_text_operators = pd.DataFrame({
    '0 | Não (n)': Facebook_Comments[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']].apply(lambda x: x.value_counts()[0]),
    '0 | Não (%)': Facebook_Comments[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']].apply(lambda x: calculate_percentage(x.value_counts()[0], len(x))),
    '1 | Sim (n)': Facebook_Comments[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']].apply(lambda x: x.value_counts()[1]),
    '1 | Sim (%)': Facebook_Comments[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']].apply(lambda x: calculate_percentage(x.value_counts()[1], len(x)))
})

df_comment_text_operators.index = [operators]
df_comment_text_operators

---

#### Matriz de % Cruzadas

In [86]:
# Simplificar a página da 'DIGI News'
Facebook_Posts.loc[Facebook_Posts['page'].str.contains('DIGI'), 'page'] = 'DIGI News'
Facebook_Comments.loc[Facebook_Comments['page'].str.contains('DIGI'), 'page'] = 'DIGI News'

In [87]:
# Agrupar os dados pela variável 'page' e obter a contagem de posts que referem cada operadora em cada página
page_posts_by_operator = Facebook_Posts.groupby('page')[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']].sum().reset_index()
page_posts_by_operator = page_posts_by_operator.loc[page_posts_by_operator['page'].isin(['Vodafone', 'MEO', 'NOS', 'DIGI News'])]

# Calcular a soma total de posts para cada página
total_posts = Facebook_Posts[Facebook_Posts['page'].isin(['Vodafone', 'MEO', 'NOS', 'DIGI News'])].groupby('page').size()

# Preencher a variável 'total_posts' com os valores correspondentes
page_posts_by_operator['total_posts'] = page_posts_by_operator['page'].map(total_posts)

# Calcular as porcentagens de referências a cada operadora em relação ao total de posts em cada página
for operator in ['Vodafone', 'MEO', 'NOS', 'DIGI']:
    page_posts_by_operator['percentage_' + operator] = round((page_posts_by_operator['post_text_' + operator] / page_posts_by_operator['total_posts']) * 100, 1)

# Page Posts Matrix (n)
page_posts_matrix_n = page_posts_by_operator.set_index('page')[['post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI']]

# Adicionar linha final com o total do número de posts em cada coluna
page_posts_matrix_n.loc['Total'] = page_posts_matrix_n.sum()

page_posts_matrix_n.columns = ['Vodafone', 'MEO', 'NOS', 'DIGI News']
page_posts_matrix_n = page_posts_matrix_n.reindex(['Vodafone', 'MEO', 'NOS', 'DIGI News', 'Total'])
page_posts_matrix_n.columns.name = 'Texto c/'
page_posts_matrix_n.index.name = 'Página'

# Selecionar apenas as colunas relevantes | Page Posts Matrix (%)
page_posts_matrix = page_posts_by_operator.set_index('page')[['percentage_Vodafone', 'percentage_MEO', 'percentage_NOS', 'percentage_DIGI']]

page_posts_matrix.columns = ['Vodafone', 'MEO', 'NOS', 'DIGI News']
page_posts_matrix.columns.name = 'Texto c/'
page_posts_matrix.index.name = 'Página'
page_posts_matrix = page_posts_matrix.reindex(['Vodafone', 'MEO', 'NOS', 'DIGI News'])

display_side_by_side(page_posts_matrix_n, page_posts_matrix,
                     super_title="Matriz de % Horizontais de Referências a Operadoras em Posts <br></br>",
                     titles=["n", "%"])

---

In [88]:
# Agrupar os dados pela variável 'page' e obter a contagem de comentários que referem cada operadora em cada página
page_comments_by_operator = Facebook_Comments.groupby('page')[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']].sum().reset_index()
page_comments_by_operator = page_comments_by_operator.loc[page_comments_by_operator['page'].isin(['Vodafone', 'MEO', 'NOS', 'DIGI News'])]

# Calcular a soma total de comentários para cada página
total_comments = Facebook_Comments[Facebook_Comments['page'].isin(['Vodafone', 'MEO', 'NOS', 'DIGI News'])].groupby('page').size()

# Preencher a variável 'total_comments' com os valores correspondentes
page_comments_by_operator['total_comments'] = page_comments_by_operator['page'].map(total_comments)

# Calcular as porcentagens de referências a cada operadora em relação ao total de comentários em cada página
for operator in ['Vodafone', 'MEO', 'NOS', 'DIGI']:
    page_comments_by_operator['percentage_' + operator] = round((page_comments_by_operator['comment_text_' + operator] / page_comments_by_operator['total_comments']) * 100,1)

# Page Comments Matrix (n)
page_comments_matrix_n = page_comments_by_operator.set_index('page')[['comment_text_Vodafone', 'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI']]

# Adicionar linha final com o total do número de comentários em cada coluna
page_comments_matrix_n.loc['Total'] = page_comments_matrix_n.sum()

page_comments_matrix_n.columns = ['Vodafone', 'MEO', 'NOS', 'DIGI News']
page_comments_matrix_n.columns.name = 'Texto c/'
page_comments_matrix_n = page_comments_matrix_n.reindex(['Vodafone', 'MEO', 'NOS', 'DIGI News', 'Total'])
page_comments_matrix_n.index.name = 'Página'

# Selecionar apenas as colunas relevantes | Page Comments Matrix (%)
page_comments_matrix = page_comments_by_operator.set_index('page')[['percentage_Vodafone', 'percentage_MEO', 'percentage_NOS', 'percentage_DIGI']]
page_comments_matrix.columns = ['Vodafone', 'MEO', 'NOS', 'DIGI News']
page_comments_matrix.columns.name = 'Texto c/'
page_comments_matrix = page_comments_matrix.reindex(['Vodafone', 'MEO', 'NOS', 'DIGI News'])
page_comments_matrix.index.name = 'Página'

# Exibir a matriz de percentagens cruzadas
display_side_by_side(page_comments_matrix_n, page_comments_matrix,
                     super_title = "Matriz de % Horizontais de Referências a Operadoras em Comentários <br></br>",
                     titles=["n", "%"])

---

# <a class='anchor' id='4.2'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: transparent; 
            padding: 10px; color: white; border-radius: 300px; text-align: center;
            border: 2px solid #A30000;">
    <center><h2 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: #A30000;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>Topic Analysis</b></h2></center>
</div>

## 🟡 HuggingFace [mDeBERTa-v3-base-xnli-multilingual-nli-2mil7](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7)

In [47]:
# Defina o pipeline de classificação
model_path = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
moritz_classifier = pipeline("zero-shot-classification", model=model_path, tokenizer=model_path, device=device)

# Nota: para utlizar este modelo temos de dar moritz_classifier(sequence_to_classify, candidate_labels, multi_label=False)

# Defina as classes a classificar pelo modelo como tópicos
candidate_labels = [
    
    # Tópicos associados às Operadoras
    "Pacotes de Serviços", "Cobertura", "Velocidade", "Preços", "Qualidade", "Atendimento ao Cliente", "Concorrência", "5G",
    "Satisfação", "Rede",  "Fidelização", "Promoções", "Segurança", "Plataformas Streaming", "Festivais", "Comunicação",
    
    # Tópicos de Eventos Anuais
    "Páscoa", "Natal", "Ano Novo", "Santos Populares", "Passatempo", "Black Friday", "Regresso às Aulas",
    
    # Tópicos Desportivos
    "Futebol", "Cristiano Ronaldo", "Surf", "Outros desportos",    
    
    # Tópicos Gerais
    "Problemas da Sociedade", "Cinema", "Filme/Série", "Economia", "Saúde", "Videojogo", "Política", "Emprego", 
    "Tecnologia", "Inteligência Artificial", "Ambiente", "Educação", "Cultura", "Ciência", "Arte", "Religião", "Negócios", 
    "Sustentabilidade", "Moda", "Alimentação", "Viagens", "Família", "Guerra", "Pandemia", "Redes Sociais", "Sociedade"
    
    # "Internet", "Fibra", "TV", "Entretenimento" -> Tirei por retirarem importância a outros
]

# Defina a função de classificação
def mDeBERTa_classify_topics(texts):
    outputs = moritz_classifier([texts], candidate_labels, multi_label=True, batch_size=8)
    labels = [output['labels'] for output in outputs]
    scores = [output['scores'] for output in outputs]
    
    # Guardar os 2 tópicos 'labels' e 'scores' que dão de resultado
    return labels[0][:3], scores[0][:3]

In [48]:
# Exemplo
exemplo_topicos = "A companhia ideal para ver todas as séries vencedoras de Globos de Ouro \
e os filmes que queremos ver nomeados nos Óscares.  #VodafoneTVPLAY"

# exemplo_topicos = "Cara Vodafone, vocês sabiam que no mundo ocidental a taxa de suicídio nos homens \
# é de 3 a 4 vezes superior que nas mulheres? Quando irá aparecer uma campanha a defender e apoiar todos \
# aqueles homens que vos montam as antenas ou que trabalham horas a fi"


print("\033[1mElementos da Pipeline:\033[0m")
for key, value in moritz_classifier(exemplo_topicos, candidate_labels, multi_label=True).items():
    print(f"\033[1m{key}:\033[0m {value} \n")
    
print("\033[1m-----------------------------------------------------\033[0m")
print("\n\033[1mOutput da Função mDeBERTa_classify_sentiment:\033[0m \n", mDeBERTa_classify_topics(exemplo_topicos))

```Python
# ------------------------------ Já corri - 03.05.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos posts [Tópicos]
mDeBERTa_posts_topics = Facebook_Posts['post_text'].progress_apply(lambda x: mDeBERTa_classify_topics([x]))
top_labels, top_scores = zip(*[(label, score) for label, score in mDeBERTa_posts_topics])

# Guardar os resultados em colunas separadas
Facebook_Posts['mDeBERTa_post_topic_label_1'] = [label[0] for label in top_labels]
Facebook_Posts['mDeBERTa_post_topic_score_1'] = [score[0] for score in top_scores]
Facebook_Posts['mDeBERTa_post_topic_label_2'] = [label[1] for label in top_labels]
Facebook_Posts['mDeBERTa_post_topic_score_2'] = [score[1] for score in top_scores]
Facebook_Posts['mDeBERTa_post_topic_label_3'] = [label[2] for label in top_labels]
Facebook_Posts['mDeBERTa_post_topic_score_3'] = [score[2] for score in top_scores]

# Salvar as novas colunas como um arquivo CSV
Facebook_Posts[['post_id', 'post_text',
                'mDeBERTa_post_topic_label_1', 'mDeBERTa_post_topic_score_1',
                'mDeBERTa_post_topic_label_2', 'mDeBERTa_post_topic_score_2',
                'mDeBERTa_post_topic_label_3', 'mDeBERTa_post_topic_score_3']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_topics.csv', encoding='utf-8', index=False)
```

In [89]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_post_topics = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_topics.csv', encoding='utf-8') \
                          .drop(['post_id','post_text'], axis=1)
Facebook_Posts = Facebook_Posts.assign(**mDeBERTa_post_topics)

In [90]:
Facebook_Posts[['post_text', 'mDeBERTa_post_topic_label_1', 'mDeBERTa_post_topic_label_2', 'mDeBERTa_post_topic_label_3']].sample(5)

---

```Python
# ------------------------------ Já corri - 04.05.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos comments [Tópicos]
mDeBERTa_comments_topics = Facebook_Comments['comment_text'].progress_apply(lambda x: mDeBERTa_classify_topics([x]))
top_labels, top_scores = zip(*[(label, score) for label, score in mDeBERTa_comments_topics])

# Guardar os resultados em colunas separadas
Facebook_Comments['mDeBERTa_comment_topic_label_1'] = [label[0] for label in top_labels]
Facebook_Comments['mDeBERTa_comment_topic_score_1'] = [score[0] for score in top_scores]
Facebook_Comments['mDeBERTa_comment_topic_label_2'] = [label[1] for label in top_labels]
Facebook_Comments['mDeBERTa_comment_topic_score_2'] = [score[1] for score in top_scores]
Facebook_Comments['mDeBERTa_comment_topic_label_3'] = [label[2] for label in top_labels]
Facebook_Comments['mDeBERTa_comment_topic_score_3'] = [score[2] for score in top_scores]

# Salvar as novas colunas como um arquivo CSV
Facebook_Comments[['comment_id', 'comment_text',
                'mDeBERTa_comment_topic_label_1', 'mDeBERTa_comment_topic_score_1',
                'mDeBERTa_comment_topic_label_2', 'mDeBERTa_comment_topic_score_2',
                'mDeBERTa_comment_topic_label_3', 'mDeBERTa_comment_topic_score_3']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_topics.csv', encoding='utf-8', index=False)
```

In [91]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_comment_topics = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_topics.csv', encoding='utf-8') \
                            .drop(['comment_id','comment_text'], axis=1)
Facebook_Comments = Facebook_Comments.assign(**mDeBERTa_comment_topics)

In [92]:
Facebook_Comments[['comment_text', 'mDeBERTa_comment_topic_label_1', 'mDeBERTa_comment_topic_label_2', 
                   'mDeBERTa_comment_topic_label_3']].sample(5)

### 👁️‍🗨️Teste do Algoritmo

In [95]:
# pd.set_option('display.max_rows', None) # Ver as linhas todas

In [24]:
# # ------------------------------ TESTE ------------------------------
# # Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos posts [Tópicos]
# mDeBERTa_posts_topics = Facebook_Posts['post_text'].progress_apply(lambda x: mDeBERTa_classify_topics([x]))
# top_labels, top_scores = zip(*[(label, score) for label, score in mDeBERTa_posts_topics])

# pd.DataFrame({
#     'post_text': Facebook_Posts['post_text'],
#     'mDeBERTa_post_topic_label_1': [label[0] for label in top_labels],
#     'mDeBERTa_post_topic_score_1' : [score[0] for score in top_scores],
#     'mDeBERTa_post_topic_label_2': [label[1] for label in top_labels],
#     'mDeBERTa_post_topic_score_2' : [score[1] for score in top_scores],
#     'mDeBERTa_post_topic_label_3': [label[2] for label in top_labels],
#     'mDeBERTa_post_topic_score_3' : [score[2] for score in top_scores],
# })

# # # Guardar os resultados em colunas separadas
# # Facebook_Posts['mDeBERTa_post_topic_label_1'] = [label[0] for label in top_labels]
# # Facebook_Posts['mDeBERTa_post_topic_score_1'] = [score[0] for score in top_scores]
# # Facebook_Posts['mDeBERTa_post_topic_label_2'] = [label[1] for label in top_labels]
# # Facebook_Posts['mDeBERTa_post_topic_score_2'] = [score[1] for score in top_scores]

# # # Salvar as novas colunas como um arquivo CSV
# # Facebook_Posts[['post_id', 'post_text',
# #                 'mDeBERTa_post_topic_label_1', 'mDeBERTa_post_topic_score_1',
# #                 'mDeBERTa_post_topic_label_2', 'mDeBERTa_post_topic_score_2']] \
# #     .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_topics.csv', encoding='utf-8', index=False)

In [176]:
# # Define a semente (seed) para garantir reprodutibilidade
# seed = 42

# # Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos posts [Tópicos]
# mDeBERTa_posts_topics =  Facebook_Posts.sample(n=100, random_state=seed)['post_text'].progress_apply(lambda x: mDeBERTa_classify_topics([x]))
# top_labels, top_scores = zip(*[(label, score) for label, score in mDeBERTa_posts_topics])

# # Criar o DataFrame com os resultados
# pd.DataFrame({
#     'post_text': Facebook_Posts.sample(n=100, random_state=seed)['post_text'],
#     'mDeBERTa_post_topic_label_1': [label[0] for label in top_labels],
#     'mDeBERTa_post_topic_score_1' : [score[0] for score in top_scores],
#     'mDeBERTa_post_topic_label_2': [label[1] for label in top_labels],
#     'mDeBERTa_post_topic_score_2' : [score[1] for score in top_scores],
#     'mDeBERTa_post_topic_label_3': [label[2] for label in top_labels],
#     'mDeBERTa_post_topic_score_3' : [score[2] for score in top_scores],
#     'mDeBERTa_post_topic_label_4': [label[3] for label in top_labels],
#     'mDeBERTa_post_topic_score_4' : [score[3] for score in top_scores],
#     'mDeBERTa_post_topic_label_5': [label[4] for label in top_labels],
#     'mDeBERTa_post_topic_score_5' : [score[4] for score in top_scores],
#     'mDeBERTa_post_topic_label_6': [label[5] for label in top_labels],
#     'mDeBERTa_post_topic_score_6' : [score[5] for score in top_scores],
#     'mDeBERTa_post_topic_label_7': [label[6] for label in top_labels],
#     'mDeBERTa_post_topic_score_7' : [score[6] for score in top_scores],
#     'mDeBERTa_post_topic_label_8': [label[7] for label in top_labels],
#     'mDeBERTa_post_topic_score_8' : [score[7] for score in top_scores],
#     'mDeBERTa_post_topic_label_9': [label[8] for label in top_labels],
#     'mDeBERTa_post_topic_score_9' : [score[8] for score in top_scores],
#     'mDeBERTa_post_topic_label_10': [label[9] for label in top_labels],
#     'mDeBERTa_post_topic_score_10' : [score[9] for score in top_scores]
# })

---

## 🖇️ Concatenar Tópicos

### 🔵 Extra | Carece de Resposta?

In [28]:
############################################ [Carecia de Resposta]

# Defina o pipeline de classificação
model_path = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
moritz_classifier = pipeline("zero-shot-classification", model=model_path, tokenizer=model_path, device=device)
candidate_labels = ["Carece de Resposta", "Não Carece de Resposta"]

# Defina a função de classificação
def mDeBERTa_classify_CR_NCR(texts):
    outputs = moritz_classifier([texts], candidate_labels, multi_label=False, batch_size=8)
    labels = [output['labels'] for output in outputs]
    scores = [output['scores'] for output in outputs]
    
    # Guardar os 2 tópicos 'labels' e 'scores' que dão de resultado
    return labels[0][0], scores[0][0]

In [29]:
# Exemplo
exemplo_CR_NCR = 'Força FCP'
# exemplo_CR_NCR = 'Serviço de Baixa Qualidade!'

print("\033[1mTexto Exemplo:\033[0m", exemplo_CR_NCR)
print("\n\033[1mOutput da Função mDeBERTa_classify_CR_NCR:\033[0m \n", mDeBERTa_classify_CR_NCR(exemplo_CR_NCR))

```Python
# ------------------------------ Já corri - 05.05.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos posts [Carece de Resposta]
mDeBERTa_posts_CR = Facebook_Posts['post_text'].progress_apply(lambda x: mDeBERTa_classify_CR_NCR([x]))
CR_labels, CR_scores = zip(*[(label, score) for label, score in mDeBERTa_posts_CR])

# Guardar os resultados em colunas separadas
Facebook_Posts['mDeBERTa_post_CR_label'] = CR_labels
Facebook_Posts['mDeBERTa_post_CR_score'] = CR_scores

# Salvar as novas colunas como um arquivo CSV
Facebook_Posts[['post_id', 'post_text',
                'mDeBERTa_post_CR_label', 'mDeBERTa_post_CR_score']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_CR.csv', encoding='utf-8', index=False)
```

In [93]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_post_CR = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_post_CR.csv', encoding='utf-8') \
                          .drop(['post_id','post_text'], axis=1)
Facebook_Posts = Facebook_Posts.assign(**mDeBERTa_post_CR)

In [94]:
Facebook_Posts[['post_text', 'mDeBERTa_post_CR_label', 'mDeBERTa_post_CR_score']].sample(5)

In [123]:
# Dataframe com a contagem (n) e percentagem (%) dos casos que Carecem ou Não Resposta
CR_posts_df = pd.DataFrame({
    'n':Facebook_Posts['mDeBERTa_post_CR_label'].value_counts(),
    '%':round(Facebook_Posts['mDeBERTa_post_CR_label'].value_counts(normalize=True) * 100,1)
})
CR_posts_df.index.name = None

# Filtra as linhas em que o mDeBERTa_post_CR_label é "Carece de Resposta" e o mDeBERTa_post_CR_score é superior a 0.9
filtered_posts = Facebook_Posts[(Facebook_Posts['mDeBERTa_post_CR_label'] == 'Carece de Resposta') &
                                (Facebook_Posts['mDeBERTa_post_CR_score'] > 0.8)]

# Cria um DataFrame com a contagem e a porcentagem de posts que carecem de resposta
CR_posts_threshold_df = pd.DataFrame({
    'n': filtered_posts['mDeBERTa_post_CR_label'].value_counts(),
    '%': round((filtered_posts['mDeBERTa_post_CR_label'].value_counts().sum() / len(Facebook_Posts) * 100), 1)
})

# Cria um DataFrame com a contagem e a porcentagem de posts que não carecem de resposta
not_CR_posts_threshold_df = pd.DataFrame({
    'n': len(Facebook_Posts) - filtered_posts['mDeBERTa_post_CR_label'].value_counts().sum(),
    '%': round((len(Facebook_Posts) - filtered_posts['mDeBERTa_post_CR_label'].value_counts().sum()) / len(Facebook_Posts) * 100, 1)
}, index=['Não Carece de Resposta'])

# Concatena os dois DataFrames
CR_posts_threshold_df = pd.concat([CR_posts_threshold_df, not_CR_posts_threshold_df])

display_side_by_side(CR_posts_df.sort_index() ,CR_posts_threshold_df,
                     titles=['Original', 'Threshold = 0.9'],
                     super_title= 'Carece de Resposta? <br><br><br>')

---

```Python
# ------------------------------ Já corri - 05.05.2024 ------------------------------
# Aplicar o mDeBERTa-v3-base-mnli-xnli ao dataset dos comments [Carece de Resposta]
mDeBERTa_comments_CR = Facebook_Comments['comment_text'].progress_apply(lambda x: mDeBERTa_classify_CR_NCR([x]))
CR_labels, CR_scores = zip(*[(label, score) for label, score in mDeBERTa_comments_CR])

# Guardar os resultados em colunas separadas
Facebook_Comments['mDeBERTa_comment_CR_label'] = CR_labels
Facebook_Comments['mDeBERTa_comment_CR_score'] = CR_scores

# Salvar as novas colunas como um arquivo CSV
Facebook_Comments[['comment_id', 'comment_text',
                   'mDeBERTa_comment_CR_label', 'mDeBERTa_comment_CR_score']] \
    .to_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_CR.csv', encoding='utf-8', index=False)
```

In [96]:
# Ler o arquivo CSV e adicionar a coluna ao DataFrame original
mDeBERTa_comment_CR = pd.read_csv('Datasets_Vodafone/TextMining/mDeBERTa_comment_CR.csv', encoding='utf-8') \
                        .drop(['comment_id','comment_text'], axis=1)
Facebook_Comments = Facebook_Comments.assign(**mDeBERTa_comment_CR)

In [97]:
Facebook_Comments[['post_text', 'mDeBERTa_comment_CR_label', 'mDeBERTa_comment_CR_score']].sample(5)

In [175]:
# Dataframe com a contagem (n) e percentagem (%) dos casos que Carecem ou Não Resposta
CR_comments_df = pd.DataFrame({
    'n':Facebook_Comments['mDeBERTa_comment_CR_label'].value_counts(),
    '%':round(Facebook_Comments['mDeBERTa_comment_CR_label'].value_counts(normalize=True) * 100,1)
})
CR_comments_df.index.name = None

# Filtra as linhas em que o mDeBERTa_comment_CR_label é "Carece de Resposta" e o mDeBERTa_comment_CR_score é superior a 0.9
filtered_comments = Facebook_Comments[(Facebook_Comments['mDeBERTa_comment_CR_label'] == 'Carece de Resposta') &
                                      (Facebook_Comments['mDeBERTa_comment_CR_score'] > 0.8)]

# Cria um DataFrame com a contagem e a porcentagem de comentários que carecem de resposta
CR_comments_threshold_df = pd.DataFrame({
    'n': filtered_comments['mDeBERTa_comment_CR_label'].value_counts(),
    '%': round((filtered_comments['mDeBERTa_comment_CR_label'].value_counts().sum() / len(Facebook_Comments) * 100), 1)
})

# Cria um DataFrame com a contagem e a porcentagem de comentários que não carecem de resposta
not_CR_comments_threshold_df = pd.DataFrame({
    'n': len(Facebook_Comments) - filtered_comments['mDeBERTa_comment_CR_label'].value_counts().sum(),
    '%': round((len(Facebook_Comments) - filtered_comments['mDeBERTa_comment_CR_label'].value_counts().sum()) / len(Facebook_Comments) * 100, 1)
}, index=['Não Carece de Resposta'])

# Concatena os dois DataFrames
CR_comments_threshold_df = pd.concat([CR_comments_threshold_df, not_CR_comments_threshold_df])

display_side_by_side(CR_comments_df.sort_index() ,CR_comments_threshold_df,
                     titles=['Original', 'Threshold = 0.8'],
                     super_title= 'Carece de Resposta? <br><br><br>')

---

### 💾 Guardar Bases de Dados com Análise

In [137]:
# Colunas repetidas entre datasets
print(list(Facebook_Posts.columns.intersection(Facebook_Comments.columns)))

In [127]:
# Compilar as bases de dados
Facebook_Posts_Comments = pd.merge(Facebook_Posts, 
                                   Facebook_Comments.drop(['post_text', 'post_date', 'post_reactions','post_comments', 
                                                           'post_shares', 'post_day', 'post_month', 'post_year', 
                                                           'post_hour', 'post_link', 'page'], axis=1), 
                                   how = 'right', on = ['post_id'])

Facebook_PCU_Analysis = pd.merge(Facebook_Posts_Comments, 
                                 Facebook_Users.drop(['user_name'], axis=1), how='left', on=['user_link'])

In [132]:
# Reorganizar as colunas
colunas_organizadas = [
    # Variáveis dos Posts
    'page', 'post_id', 'post_link', 'post_date', 'post_day', 'post_month', 'post_year', 'post_hour',
    'post_reactions', 'post_comments', 'post_shares', 'post_text', 'post_text_clean', 'post_language', 
    'post_sentiment_label','TXRBSF_post_sentiment_label', 'TXRBSF_post_sentiment_score', 
    'mDeBERTa_post_sentiment_label', 'mDeBERTa_post_sentiment_score', 
    'post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI',
    'mDeBERTa_post_topic_label_1', 'mDeBERTa_post_topic_score_1', 'mDeBERTa_post_topic_label_2',
    'mDeBERTa_post_topic_score_2', 'mDeBERTa_post_topic_label_3', 'mDeBERTa_post_topic_score_3',
    'mDeBERTa_post_CR_label', 'mDeBERTa_post_CR_score',
    
    # Variáveis dos Comments
    'comment_id', 'comment_link', 'comment_date', 'comment_day_ago', 'comment_reactions',
    'comment_num_responses', 'comment_operator_responded', 'comment_text', 'comment_text_clean', 'comment_language',
    'comment_sentiment_label', 'TXRBSF_comment_sentiment_label', 'TXRBSF_comment_sentiment_score',
    'mDeBERTa_comment_sentiment_label', 'mDeBERTa_comment_sentiment_score', 'comment_text_Vodafone',
    'comment_text_MEO', 'comment_text_NOS', 'comment_text_DIGI',
    'mDeBERTa_comment_topic_label_1', 'mDeBERTa_comment_topic_score_1',
    'mDeBERTa_comment_topic_label_2', 'mDeBERTa_comment_topic_score_2',
    'mDeBERTa_comment_topic_label_3', 'mDeBERTa_comment_topic_score_3',
    'mDeBERTa_comment_CR_label', 'mDeBERTa_comment_CR_score',

    # Variáveis dos Users
    'user_name', 'user_link', 'user_current_city', 'user_hometown', 'user_freguesia', 'user_concelho',
    'user_distrito', 'user_pais', 'user_city_not_portugal', 'user_predicted_genre'
]

Facebook_PCU_Analysis = Facebook_PCU_Analysis[colunas_organizadas]

```Python
# Guardar o dataset final com toda a análise em formato .txt
Facebook_PCU_Analysis.to_csv('Datasets_Vodafone/Facebook_PCU_Analysis.txt', sep='\t', index=False, encoding='utf-8')
```

---

# <a class='anchor' id='5'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>5 | Evaluation</b></h1></center>
</div>

In [5]:
# Import da Base de Dados com Análise
Facebook_PCU_Analysis = pd.read_csv('Datasets_Vodafone/Facebook_PCU_Analysis.txt', sep='\t', encoding='utf-8')

In [6]:
# Criar o Facebook_Posts_Analysis
Facebook_Posts_Analysis = Facebook_PCU_Analysis.groupby('post_id').first().reset_index()

# Selecionar apenas as colunas relevantes para o Facebook_Posts_Analysis
Facebook_Posts_Analysis = Facebook_Posts_Analysis[['post_id', 'page', 'post_link', 'post_date', 'post_day', 'post_month', 
                                                   'post_year', 'post_hour', 'post_reactions', 'post_comments', 'post_shares',
                                                   'post_text', 'post_text_clean', 'post_language', 'post_sentiment_label',
                                                   'TXRBSF_post_sentiment_label', 'TXRBSF_post_sentiment_score', 
                                                   'mDeBERTa_post_sentiment_label', 'mDeBERTa_post_sentiment_score', 
                                                   'post_text_Vodafone', 'post_text_MEO', 'post_text_NOS', 'post_text_DIGI', 
                                                   'mDeBERTa_post_topic_label_1', 'mDeBERTa_post_topic_score_1', 
                                                   'mDeBERTa_post_topic_label_2', 'mDeBERTa_post_topic_score_2', 
                                                   'mDeBERTa_post_topic_label_3', 'mDeBERTa_post_topic_score_3', 
                                                   'mDeBERTa_post_CR_label', 'mDeBERTa_post_CR_score']]

# Criar o Facebook_Comments_Analysis
Facebook_Comments_Analysis = Facebook_PCU_Analysis.groupby('comment_id').first().reset_index()

---

## ⚖️ Divisão em Conjunto Treino/Teste

In [7]:
from sklearn import model_selection

# Dividir a amostra em duas partes: 60% para treinamento e 40% para teste
dtf_train, dtf_test = model_selection.train_test_split(Facebook_Comments_Analysis, test_size=0.4, random_state=123)

# Tamanho dos dataset de Treino e Teste
print("\033[1mConjunto de Treino:\033[0m", dtf_train.shape)
print("\033[1mConjunto de Teste:\033[0m", dtf_test.shape)

## 🛠️ Análise dos Resultados

### 🔴🟡🟢 Sentimentos

In [8]:
# Criar um DataFrame com as contagens e percentagens das labels do treino e teste
labels_df = pd.DataFrame()

# Contagens e percentagens das labels do treino para post_sentiment_label
# labels_df['Treino post_sentiment_label (n)'] = dtf_train['post_sentiment_label'].value_counts()
labels_df['Treino post_sentiment_label (%)'] = round(dtf_train['post_sentiment_label'].value_counts(normalize=True) * 100,1)

# Contagens e percentagens das labels do teste para post_sentiment_label
# labels_df['Teste post_sentiment_label (n)'] = dtf_test['post_sentiment_label'].value_counts()
labels_df['Teste post_sentiment_label (%)'] = round(dtf_test['post_sentiment_label'].value_counts(normalize=True) * 100,1)

labels_df[' '] = ['', '', '', '', '']

# Contagens e percentagens das labels do treino para comment_sentiment_label
# labels_df['Treino comment_sentiment_label (n)'] = dtf_train['comment_sentiment_label'].value_counts()
labels_df['Treino comment_sentiment_label (%)'] = round(dtf_train['comment_sentiment_label'].value_counts(normalize=True) * 100,1)

# Contagens e percentagens das labels do teste para comment_sentiment_label
# labels_df['Teste comment_sentiment_label (n)'] = dtf_test['comment_sentiment_label'].value_counts()
labels_df['Teste comment_sentiment_label (%)'] = round(dtf_test['comment_sentiment_label'].value_counts(normalize=True) * 100,1)

labels_df.T

---

In [9]:
# Criar um gráfico de barras com as contagens das labels do treino e teste lado a lado
fig, ax = plt.subplots(figsize=(12,5))

# Criar uma lista de 4 cores diferentes para treino e teste
colors = ['#5B1C89', '#CE9BF3', '#094A9E', '#5D9CEE']

# Ordenar as barras por valor
labels_df.plot(kind='bar', 
               color=colors, 
               ax=ax)

# Adicionar rótulos dos eixos e título
plt.xlabel('')
plt.ylabel('n\n')
plt.title('Contagem de Labels - Treino vs Teste \n', fontsize=18, fontweight='bold')

# Adicionar legenda
legend_properties = {'weight':'bold', 'size':'14'}
ax.legend(['Treino Posts', 'Teste Post', 'Treino Comments', 'Teste Comments'], 
          title='Conjuntos', title_fontproperties=legend_properties, fontsize='12', loc='upper right',frameon=False)

# Ajustar os rótulos dos nomes das labels no eixo x
plt.xticks(rotation=0, ha='center', rotation_mode='anchor')

sns.despine(top=True, right=True)
plt.tight_layout()
# fig.savefig("Relatório/Gráficos/8_Barras Resposta da Operadora.svg",format='svg',dpi=1200)
plt.show()

---

### 🔢 Teste Estatístico

Seja $S/T$ os $Sentimentos$ e $Tópicos$ em estudo e $D_1$ o *dataset* de treino e $D_2$ de teste:

- $X = D_1^{\;S/T}$: Sentimentos/Tópicos no dataset de **Treino**

- $Y = D_2^{\;S/T}$  Sentimentos/Tópicos no dataset de **Teste**

#### Hipóteses em teste

- $H_0:$ O resultados do treino **são independentes** dos de teste [$D_1^{S/T}\; \neq \;D_2^{S/T}$]

- $H_1:$ Existe relacionamento entre os resultados de treino e teste [$D_1^{S/T}\; = \;D_2^{S/T}$]

ou, teoricamente,

- $H_0:\forall(i, j) \in\{1: r\} \times\{1: c\}: p_{i j}=p_{i .} \times p_{. j}$

- $H_1: \exists(i, j) \in\{1: r\} \times\{1: c\}: p_{i j} \neq p_{i .} \times p_{\cdot j}$

---

#### Teste Qui-Quadrado

**Estatística de teste**

$$E T=\sum_{i=1}^r \sum_{j=1}^c \frac{\left(o_{i j}-e_{i j}\right)^2}{e_{i j}} \; \dot\sim \; \chi_{(r-1)(c-1)}^2$$

Sendo ambas variáveis fatores, o estudo do relacionamento entre elas será feito através da análise do respetivo
cruzamento (tabela de contingência), com a subsequente aplicação do **Teste Qui-quadrado**.

In [10]:
import scipy.stats as stats

# Crie um DataFrame com as contagens das labels do treino e teste
chi2_df = pd.DataFrame()

# Contagens das labels do treino para comment_sentiment_label
chi2_df['Treino comment_sentiment_label'] = dtf_train['comment_sentiment_label'].dropna().value_counts()

# Contagens das labels do teste para comment_sentiment_label
chi2_df['Teste comment_sentiment_label'] = dtf_test['comment_sentiment_label'].dropna().value_counts()

# Crie uma tabela de contingência com as contagens das labels do treino e teste
contingency_table = chi2_df.values

# Realize o teste qui-quadrado
chi2, p, dof, expected = stats.chi2_contingency(contingency_table, correction=False)

print('\033[1mEstatística qui-quadrado:\033[0m {:.4f}'.format(chi2))
print('\033[1mValor-p:\033[0m {:.4f}'.format(p))
print('\033[1mGraus de liberdade:\033[0m {}'.format(dof))
print('\033[1mTabela de contingência esperada:\033[0m\n')
pd.DataFrame(expected, index=chi2_df.index , columns=['Treino', 'Teste'])


# No exemplo acima, a estatística qui-quadrado é igual a 6,83, o valor-p é igual a 0,033 e os graus de liberdade = 2. 
# Como o valor-p é menor que o nível de significância usual de 0,05, você pode rejeitar a hipótese nula de que as 
# distribuições das labels do treino e teste são iguais. Isso significa que há evidências estatísticas de que as 
# distribuições das labels do treino e teste são diferentes.

> Como $p-value = 0.0079 < \alpha$ de referência ($\alpha=0.05$), então não se rejeita $H_0$.

Ou seja, **existe** evidência estatística que revele associação o treino e o teste.

- Isto significa que existem divergências significativas entre as frequências observadas e as frequências esperadas (ou seja, o que estamos a observar numa situação de independência). 

--- 

Assim, o modelo mostra-se capaz de classificar os dados sem enviesar os resultados.

[**Nota:** $\alpha = 0.05$ é com uma confiança de $95\%$]

---

In [11]:
import numpy as np
from scipy.stats import ks_2samp

# Calcule as ECDFs
ecdf_train = np.arange(1, len(dtf_train['comment_sentiment_label'].dropna().value_counts()) + 1) / len(dtf_train['comment_sentiment_label'].dropna().value_counts())
ecdf_test = np.arange(1, len(dtf_test['comment_sentiment_label'].dropna().value_counts()) + 1) / len(dtf_test['comment_sentiment_label'].dropna().value_counts())

# Realize o teste de Kolmogorov-Smirnov
statistic, p_value = ks_2samp(dtf_train['comment_sentiment_label'].dropna().value_counts(), dtf_test['comment_sentiment_label'].dropna().value_counts())

# Exiba os resultados
print("\033[1mEstatística D:\033[0m", statistic)
print("\033[1mValor-p:\033[0m", p_value)

> Se o $p-value = 1$, isso indica que não há evidências suficientes para rejeitar a hipótese nula de que as distribuições dos sentimentos nos conjuntos de treino e teste são as mesmas. Nesse caso, você não encontrou diferenças significativas entre as distribuições.

> Portanto, com base no resultado do **Teste de Kolmogorov-Smirnov**, você pode concluir que as distribuições de sentimentos nos conjuntos de treino e teste são semelhantes. Isso sugere que o modelo treinado pode estar generalizando bem para os dados de teste, em termos de distribuição de sentimentos.

---

### 🔴🔵⚫ Tópicos

In [12]:
# Criar um DataFrame com as contagens e percentagens das labels do treino e teste
labels_df = pd.DataFrame()

# Contagens e percentagens das labels do treino para mDeBERTa_post_topic_label_1
# labels_df['Treino mDeBERTa_post_topic_label_1 (n)'] = dtf_train['mDeBERTa_post_topic_label_1'].value_counts()
labels_df['Treino | post_topic_label_1 (%)'] = round(dtf_train['mDeBERTa_post_topic_label_1'].value_counts(normalize=True) * 100,1)

# Contagens e percentagens das labels do teste para mDeBERTa_post_topic_label_1
# labels_df['Teste mDeBERTa_post_topic_label_1 (n)'] = dtf_test['mDeBERTa_post_topic_label_1'].value_counts()
labels_df['Teste | post_topic_label_1 (%)'] = round(dtf_test['mDeBERTa_post_topic_label_1'].value_counts(normalize=True) * 100,1)

# Contagens e percentagens das labels do treino para mDeBERTa_comment_topic_label_1
# labels_df['Treino mDeBERTa_comment_topic_label_1 (n)'] = dtf_train['mDeBERTa_comment_topic_label_1'].value_counts()
labels_df['Treino | comment_topic_label_1 (%)'] = round(dtf_train['mDeBERTa_comment_topic_label_1'].value_counts(normalize=True) * 100,1)

# Contagens e percentagens das labels do teste para mDeBERTa_comment_topic_label_1
# labels_df['Teste mDeBERTa_comment_topic_label_1 (n)'] = dtf_test['mDeBERTa_comment_topic_label_1'].value_counts()
labels_df['Teste | comment_topic_label_1 (%)'] = round(dtf_test['mDeBERTa_comment_topic_label_1'].value_counts(normalize=True) * 100,1)

labels_df.index.name = None
labels_df

In [13]:
import matplotlib.colors as mcolors
# Criar um gráfico de barras com as contagens das labels do treino e teste lado a lado
fig, ax = plt.subplots(figsize=(15,6))

# Criar uma lista de 4 cores diferentes para treino e teste
colors = ['#5B1C89', '#CE9BF3', '#094A9E', '#5D9CEE']

# Ordenar as barras por valor
# labels_df.sort_values(by=['Treino | post_topic_label_1 (%)', 'Teste | post_topic_label_1 (%)'], ascending=False).plot(kind='bar', color=colors, ax=ax)
labels_df.sort_values(by=['Treino | post_topic_label_1 (%)', 'Teste | post_topic_label_1 (%)'], ascending=False).head(10).plot(kind='bar', color=colors, ax=ax)

# Adicionar rótulos dos eixos e título
plt.xlabel('')
plt.ylabel('Contagem\n', fontweight='bold')
plt.ylim(0, 20)
# plt.title('Contagem de Labels - Treino vs Teste', fontsize=18, fontweight='bold')
plt.title('Top 10 Labels - Treino vs Teste', fontsize=18, fontweight='bold')

# Adicionar legenda
legend_properties = {'weight':'bold', 'size':'14'}
ax.legend(['Treino Posts', 'Teste Post', 'Treino Comments', 'Teste Comments'], 
          title='Conjuntos', title_fontproperties=legend_properties, fontsize='12', loc='upper right',frameon=False)

# Ajustar os rótulos dos nomes das labels no eixo x
plt.xticks(rotation=0, ha='center', rotation_mode='anchor')
# plt.xticks(rotation=25, ha='right', rotation_mode='anchor')

sns.despine(top=True, right=True)
plt.tight_layout()
# fig.savefig("Relatório/Gráficos/8_Barras Resposta da Operadora.svg",format='svg',dpi=1200)
plt.show()

---

### 🔢 Teste Estatístico

#### Teste Qui$^2$

In [14]:
import scipy.stats as stats

# Crie um DataFrame com as contagens das labels do treino e teste
chi2_df = pd.DataFrame()

# Contagens das labels do treino para mDeBERTa_comment_topic_label_1
chi2_df['Treino mDeBERTa_comment_topic_label_1'] = dtf_train['mDeBERTa_comment_topic_label_1'].dropna().value_counts()

# Contagens das labels do teste para mDeBERTa_comment_topic_label_1
chi2_df['Teste mDeBERTa_comment_topic_label_1'] = dtf_test['mDeBERTa_comment_topic_label_1'].dropna().value_counts()

# Crie uma tabela de contingência com as contagens das labels do treino e teste
contingency_table = chi2_df.values

# Realize o teste qui-quadrado
chi2, p, dof, expected = stats.chi2_contingency(contingency_table, correction=False)

print('\033[1mEstatística qui-quadrado:\033[0m {:.4f}'.format(chi2))
print('\033[1mValor-p:\033[0m {:.4f}'.format(p))
print('\033[1mGraus de liberdade:\033[0m {}'.format(dof))
print('\033[1mTabela de contingência esperada:\033[0m\n')
pd.DataFrame(expected, index=chi2_df.index , columns=['Treino', 'Teste'])

> $=$ ao do **Sentimento**

#### Teste de Kolmogorov-Smirnov

In [15]:
import numpy as np
from scipy.stats import ks_2samp

# Calcule as ECDFs
ecdf_train = np.arange(1, len(dtf_train['mDeBERTa_comment_topic_label_1'].dropna().value_counts()) + 1) / len(dtf_train['mDeBERTa_comment_topic_label_1'].dropna().value_counts())
ecdf_test = np.arange(1, len(dtf_test['mDeBERTa_comment_topic_label_1'].dropna().value_counts()) + 1) / len(dtf_test['mDeBERTa_comment_topic_label_1'].dropna().value_counts())

# Realize o teste de Kolmogorov-Smirnov
statistic, p_value = ks_2samp(dtf_train['mDeBERTa_comment_topic_label_1'].dropna().value_counts(), dtf_test['mDeBERTa_comment_topic_label_1'].dropna().value_counts())

# Exiba os resultados
print("\033[1mEstatística D:\033[0m", statistic)
print("\033[1mValor-p:\033[0m", p_value)

> $=$ ao do **Sentimento**

---

# <a class='anchor' id='6'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right,#A30000, #F91701); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 120px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 34px; font-family: 'Avenir Next LT Pro', sans-serif;"><b>6 | Deployment</b></h1></center>
</div>

 * 📊 Dashboard [pcfad_chatmeter_streamlit.py](./Dashboard/pcfad_chatmeter_streamlit.py)

In [18]:
print(Facebook_Comments_Analysis.info())
Facebook_PCU_Analysis.columns

In [19]:
Facebook_PCU_Analysis['user_pais'].value_counts()

In [20]:
# Colunas a eliminar
colunas_del = [
    'post_day', 'post_month', 'post_year', 'post_hour', 'post_language',
    'comment_date', 'comment_day_ago', 'comment_language', 'user_current_city', 'user_hometown', 'user_city_not_portugal'
]

In [ ]:
# Eliminar colunas desnecessárias
# Facebook_PCU_Analysis = Facebook_PCU_Analysis.drop(columns=colunas_del)

# Guardar o dataset resultante como um arquivo CSV
# Facebook_PCU_Analysis.to_csv('Datasets_Vodafone/Facebook_PCU_Analysis.csv', sep='|', encoding='utf-8', index=False)

# Guardar o dataset resultante como um arquivo Pickle
# Facebook_PCU_Analysis.to_pickle('Datasets_Vodafone/Facebook_PCU_Analysis.pkl')

```Python
# Dividir o DataFrame em partes menores
num_parts = 3  # Defina o número de partes
chunk_size = len(Facebook_PCU_Analysis) // num_parts
chunks = [Facebook_PCU_Analysis[i:i + chunk_size] for i in range(0, len(Facebook_PCU_Analysis), chunk_size)]

# Salvar as partes como arquivos separados
for i, chunk in enumerate(chunks):
    chunk.to_pickle(f'Datasets_Vodafone/Facebook_PCU_Analysis_part_{i+1}.pkl')
```

---